<a href="https://colab.research.google.com/github/anju0007/Open-Payments-Database-Analyses/blob/main/Endocrinology%2C%20Diabetes%20%26%20Metabolism/Endocrinology%2C_Diabetes_%26_Metabolism_version_Diabetes_Care_R4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Payments Database analysis
# Endocrinology, Diabetes & Metabolism

# Specialty

In [ ]:
specialty = "Endocrinology, Diabetes & Metabolism"

# Import package

## import package

In [ ]:
# Check Python Version
!python --version

Python 3.10.6


In [ ]:
# Check GPU
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import os
import glob
import sys
import numpy as np
import numexpr
import pandas as pd
from tqdm import tqdm


## mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Declare path to import and save payment dataset

In [ ]:
BASE = "/content/drive/MyDrive/OPD_data out put"

In [ ]:
path_out1 = BASE + '/'  + specialty+ "/general/"
path_out2 = BASE + '/' + specialty+ "/research/"
path_out3 = BASE + '/' + specialty+ "/ownership/"
path_out_profile = BASE + '/' + specialty+ "/profile/"
path_out_stata = BASE + '/' + specialty+ "/stata/"

# NPPES specialty taxonomy

In [ ]:
special = pd.read_csv("/content/drive/MyDrive/national provider identifier file/nucc_taxonomy_221.csv",
                   low_memory=False,
                  keep_default_na= "0")

In [ ]:
special

,Code,Grouping,Classification,Specialization,Definition,Effective Date,Deactivation Date,Last Modified Date,Notes,Display Name,Section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,10/1/2003,NaN,NaN,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,10/1/2003,NaN,NaN,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,4/1/2003,NaN,7/1/2007,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,4/1/2003,NaN,NaN,NaN,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,4/1/2003,NaN,NaN,NaN,Clinical & Laboratory Immunology (Allergy & Im...,Individual
...,...,...,...,...,...,...,...,...,...,...,...
863,343800000X,Transportation Services,Secured Medical Transport (VAN),NaN,A public or privately owned transportation ser...,4/1/2002,NaN,NaN,NaN,Secured Medical Transport (VAN),Non-Individual
864,344600000X,Transportation Services,Taxi,NaN,A land commercial vehicle used for the transpo...,4/1/2002,NaN,NaN,NaN,Taxi,Non-Individual
865,347D00000X,Transportation Services,Train,NaN,An organization or business licensed to provid...,4/1/2002,NaN,NaN,NaN,Train,Non-Individual
866,347E00000X,Transportation Services,Transportation Broker,NaN,An organization that provides transportation f...,4/1/2002,NaN,1/1/2021,Source: Section 6083 of the Deficit Reduction ...,Transportation Broker,Non-Individual


In [ ]:
physician = special.query('Grouping == "Allopathic & Osteopathic Physicians"')
internal = physician[physician['Specialization']== specialty]
spe =list(internal['Code'])
spe

['207RE0101X']

# NPI database extraction

## NPI dataset

In [ ]:
usecols = ['NPI', 'Entity Type Code', 'Replacement NPI',
           'Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)',
           'Provider First Name', 'Provider Middle Name',
           'Provider Other Organization Name', 'Provider Other Organization Name Type Code',
           'Provider Other Last Name', 'Provider Other First Name',
           'Provider Other Middle Name',  'Provider Business Practice Location Address City Name',
           'Provider Business Practice Location Address State Name', 'Provider Enumeration Date',
           'Last Update Date', 'NPI Deactivation Reason Code', 'NPI Deactivation Date',
           'NPI Reactivation Date', 'Provider Gender Code', 'Authorized Official Title or Position',
           'Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1',
           'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2',
           'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3',
           'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4',
           'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5',
           'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6',
           'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7',
           'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8',
           'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9',
           'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10',
           'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11',
           'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12',
           'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13',
           'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14',
           'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15',
           'Certification Date' , 'Provider Business Practice Location Address Postal Code',
           'Provider Business Practice Location Address Country Code (If outside U.S.)']

In [ ]:
npi = pd.read_csv("/content/drive/MyDrive/NPPES_Data_Dissemination_April_2023/npidata_pfile_20050523-20230409.csv",
                    chunksize= 1000000, usecols= usecols, low_memory=False,
                    keep_default_na= "0")
npi = pd.concat((r for r in npi), ignore_index=True)
npi = npi[npi["Entity Type Code"] == 1]

In [ ]:
npi_specialty = npi[((npi["Healthcare Provider Primary Taxonomy Switch_1"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_1"].isin(spe))) |
                   ((npi["Healthcare Provider Primary Taxonomy Switch_2"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_2"].isin(spe))) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_3"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_3"].isin(spe))) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_4"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_4"].isin(spe))) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_5"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_5"].isin(spe))) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_6"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_6"].isin(spe))) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_7"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_7"].isin(spe) )) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_8"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_8"].isin(spe) )) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_9"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_9"].isin(spe) )) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_10"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_10"].isin(spe) )) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_11"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_11"].isin(spe) )) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_12"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_12"].isin(spe) )) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_13"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_13"].isin(spe) )) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_14"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_14"].isin(spe) )) |
                    ((npi["Healthcare Provider Primary Taxonomy Switch_15"] == "Y" ) & (npi["Healthcare Provider Taxonomy Code_15"].isin(spe) )) ]
del npi

In [ ]:
npi_specialty

,NPI,Entity Type Code,Replacement NPI,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,...,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Certification Date
1308,1316940141,1.0,NaN,NaN,PASSARO,MAUREEN,D,NaN,NaN,DONNELLY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1322,1609879709,1.0,NaN,NaN,FRUITERMAN,MARK,LESTER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,1396748463,1.0,NaN,NaN,KOLATKAR,NIKHEEL,SHRINIVAS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2038,1285637553,1.0,NaN,NaN,PENABAD,JESUS,L,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2333,1972506244,1.0,NaN,NaN,LOMAN,LUSIANA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7750577,1396784344,1.0,NaN,NaN,STEINSAPIR,JAIME,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/01/2022
7750619,1689678286,1.0,NaN,NaN,CAPLAN,ROBERT,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7750622,1033179940,1.0,NaN,NaN,ELLYIN,FREDRICK,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7750744,1124184858,1.0,NaN,NaN,TUCK,MICHAEL,LAWRENCE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
npi_specialty["month"] = npi_specialty["Provider Enumeration Date"].str[:2].astype(int)
npi_specialty["day"] = npi_specialty["Provider Enumeration Date"].str.slice(start=3, stop=5).astype(int)
npi_specialty["year"] = npi_specialty["Provider Enumeration Date"].str.slice(start=6, stop=10).astype(int)

<ipython-input-15-679c06d768cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npi_specialty["month"] = npi_specialty["Provider Enumeration Date"].str[:2].astype(int)
<ipython-input-15-679c06d768cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npi_specialty["day"] = npi_specialty["Provider Enumeration Date"].str.slice(start=3, stop=5).astype(int)
<ipython-input-15-679c06d768cc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [ ]:
npi_specialty

,NPI,Entity Type Code,Replacement NPI,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,...,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Certification Date,month,day,year
1308,1316940141,1.0,NaN,NaN,PASSARO,MAUREEN,D,NaN,NaN,DONNELLY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,24,2005
1322,1609879709,1.0,NaN,NaN,FRUITERMAN,MARK,LESTER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,23,2005
1408,1396748463,1.0,NaN,NaN,KOLATKAR,NIKHEEL,SHRINIVAS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,23,2005
2038,1285637553,1.0,NaN,NaN,PENABAD,JESUS,L,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,31,2005
2333,1972506244,1.0,NaN,NaN,LOMAN,LUSIANA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,31,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7750577,1396784344,1.0,NaN,NaN,STEINSAPIR,JAIME,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12/01/2022,6,5,2006
7750619,1689678286,1.0,NaN,NaN,CAPLAN,ROBERT,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,10,2005
7750622,1033179940,1.0,NaN,NaN,ELLYIN,FREDRICK,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,24,2006
7750744,1124184858,1.0,NaN,NaN,TUCK,MICHAEL,LAWRENCE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,28,2006


In [ ]:
# define January 2000 as month 1
npi_specialty["cmonth"] = ((npi_specialty["year"]-2000)*12 + npi_specialty["month"])
# Exclude physicians who were newly activated after 2014 or deactivated during 2005 and 2023
npi_specialty_full = npi_specialty

npi_specialty = npi_specialty.query('year < 2014')
list_npi = list(npi_specialty["NPI"])
npi_specialty

,NPI,Entity Type Code,Replacement NPI,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,...,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Certification Date,month,day,year,cmonth
1308,1316940141,1.0,NaN,NaN,PASSARO,MAUREEN,D,NaN,NaN,DONNELLY,...,NaN,NaN,NaN,NaN,NaN,NaN,5,24,2005,65
1322,1609879709,1.0,NaN,NaN,FRUITERMAN,MARK,LESTER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,23,2005,65
1408,1396748463,1.0,NaN,NaN,KOLATKAR,NIKHEEL,SHRINIVAS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,23,2005,65
2038,1285637553,1.0,NaN,NaN,PENABAD,JESUS,L,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,31,2005,65
2333,1972506244,1.0,NaN,NaN,LOMAN,LUSIANA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,31,2005,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7750577,1396784344,1.0,NaN,NaN,STEINSAPIR,JAIME,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12/01/2022,6,5,2006,78
7750619,1689678286,1.0,NaN,NaN,CAPLAN,ROBERT,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6,10,2005,66
7750622,1033179940,1.0,NaN,NaN,ELLYIN,FREDRICK,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,24,2006,75
7750744,1124184858,1.0,NaN,NaN,TUCK,MICHAEL,LAWRENCE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12,28,2006,84


In [ ]:
npi_specialty_full

,NPI,Entity Type Code,Replacement NPI,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,...,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Certification Date,month,day,year,cmonth
1308,1316940141,1.0,NaN,NaN,PASSARO,MAUREEN,D,NaN,NaN,DONNELLY,...,NaN,NaN,NaN,NaN,NaN,NaN,5,24,2005,65
1322,1609879709,1.0,NaN,NaN,FRUITERMAN,MARK,LESTER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,23,2005,65
1408,1396748463,1.0,NaN,NaN,KOLATKAR,NIKHEEL,SHRINIVAS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,23,2005,65
2038,1285637553,1.0,NaN,NaN,PENABAD,JESUS,L,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,31,2005,65
2333,1972506244,1.0,NaN,NaN,LOMAN,LUSIANA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,31,2005,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7750577,1396784344,1.0,NaN,NaN,STEINSAPIR,JAIME,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12/01/2022,6,5,2006,78
7750619,1689678286,1.0,NaN,NaN,CAPLAN,ROBERT,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6,10,2005,66
7750622,1033179940,1.0,NaN,NaN,ELLYIN,FREDRICK,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,24,2006,75
7750744,1124184858,1.0,NaN,NaN,TUCK,MICHAEL,LAWRENCE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12,28,2006,84


## Merge taxonomy code and specialty name

In [ ]:
special = special.fillna("NA")

spe = list(special['Specialization'])

In [ ]:
npi_specialty = npi_specialty.assign(cla = "" ,spec = "", note = "")
npi_specialty

,NPI,Entity Type Code,Replacement NPI,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,...,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Certification Date,month,day,year,cmonth,cla,spec,note
1308,1316940141,1.0,NaN,NaN,PASSARO,MAUREEN,D,NaN,NaN,DONNELLY,...,NaN,NaN,NaN,5,24,2005,65,,,
1322,1609879709,1.0,NaN,NaN,FRUITERMAN,MARK,LESTER,NaN,NaN,NaN,...,NaN,NaN,NaN,5,23,2005,65,,,
1408,1396748463,1.0,NaN,NaN,KOLATKAR,NIKHEEL,SHRINIVAS,NaN,NaN,NaN,...,NaN,NaN,NaN,5,23,2005,65,,,
2038,1285637553,1.0,NaN,NaN,PENABAD,JESUS,L,NaN,NaN,NaN,...,NaN,NaN,NaN,5,31,2005,65,,,
2333,1972506244,1.0,NaN,NaN,LOMAN,LUSIANA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5,31,2005,65,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7750577,1396784344,1.0,NaN,NaN,STEINSAPIR,JAIME,NaN,NaN,NaN,NaN,...,NaN,NaN,12/01/2022,6,5,2006,78,,,
7750619,1689678286,1.0,NaN,NaN,CAPLAN,ROBERT,H,NaN,NaN,NaN,...,NaN,NaN,NaN,6,10,2005,66,,,
7750622,1033179940,1.0,NaN,NaN,ELLYIN,FREDRICK,M,NaN,NaN,NaN,...,NaN,NaN,NaN,3,24,2006,75,,,
7750744,1124184858,1.0,NaN,NaN,TUCK,MICHAEL,LAWRENCE,NaN,NaN,NaN,...,NaN,NaN,NaN,12,28,2006,84,,,


In [ ]:
npi_specialty.groupby(['spec'])['NPI'].count()

spec
    8002
Name: NPI, dtype: int64

## Match NPI with OPD

In [ ]:
df_profile = pd.read_csv("/content/drive/MyDrive/OPD_g/OPD_physician dataset20230630.csv", low_memory=False)

In [ ]:
df_profile['covered_recipient_npi'] = df_profile['covered_recipient_npi'].fillna(0)
df_profile['covered_recipient_npi'] = df_profile['covered_recipient_npi'].astype('int64')

In [ ]:
df_id = df_profile[df_profile["covered_recipient_npi"].isin(list_npi)]
df_id

,covered_recipient_profile_type,covered_recipient_profile_id,covered_recipient_npi,covered_recipient_profile_first_name,covered_recipient_profile_middle_name,covered_recipient_profile_last_name,covered_recipient_profile_suffix,covered_recipient_profile_alternate_first_name1,covered_recipient_profile_alternate_middle_name1,covered_recipient_profile_alternate_last_name1,...,covered_recipient_profile_license_state_code_5,covered_recipient_profile_alternate_first_name2,covered_recipient_profile_alternate_last_name2,covered_recipient_profile_alternate_first_name3,covered_recipient_profile_alternate_last_name3,covered_recipient_profile_alternate_first_name4,covered_recipient_profile_alternate_last_name4,covered_recipient_profile_alternate_first_name5,covered_recipient_profile_alternate_last_name5,has_multiple_ids
3,Covered Recipient Physician,4,1003024811,LISA,L,HAMAKER,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
36,Covered Recipient Physician,39,1003042169,JACQUELINE,Y,LONIER,NaN,JACQUELINE,YUEY,LONIER,...,NaN,NaN,NaN,JACKIE,LONIER,NaN,NaN,NaN,NaN,False
59,Covered Recipient Physician,64,1003081944,TONSLYN,A,TOURE,NaN,TONSLYN,NaN,TOURE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
178,Covered Recipient Physician,196,1003832437,LAWRENCE,FAYEZ,NESHIWAT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
189,Covered Recipient Physician,207,1003839101,THOMAS,M,FLOOD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296428,Covered Recipient Physician/Covered Recipient ...,11042754,1891786422,KIMBERLEE,J,BRONTSEMA,NaN,NaN,NaN,NaN,...,NaN,ROBERT,TOGNACCI,NaN,NaN,NaN,NaN,NaN,NaN,False
1337978,Covered Recipient Physician/Covered Recipient ...,11161690,1992133086,THERESA,J,FRIMEL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1361701,Covered Recipient Physician/Covered Recipient ...,11230513,1245345180,CARLA,J,CULHANE,NaN,CARLA,J,GAROFOLO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1388843,Covered Recipient Physician,11310124,1750623849,LIANG,NaN,XUE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
df_id

,covered_recipient_profile_type,covered_recipient_profile_id,covered_recipient_npi,covered_recipient_profile_first_name,covered_recipient_profile_middle_name,covered_recipient_profile_last_name,covered_recipient_profile_suffix,covered_recipient_profile_alternate_first_name1,covered_recipient_profile_alternate_middle_name1,covered_recipient_profile_alternate_last_name1,...,covered_recipient_profile_license_state_code_5,covered_recipient_profile_alternate_first_name2,covered_recipient_profile_alternate_last_name2,covered_recipient_profile_alternate_first_name3,covered_recipient_profile_alternate_last_name3,covered_recipient_profile_alternate_first_name4,covered_recipient_profile_alternate_last_name4,covered_recipient_profile_alternate_first_name5,covered_recipient_profile_alternate_last_name5,has_multiple_ids
3,Covered Recipient Physician,4,1003024811,LISA,L,HAMAKER,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
36,Covered Recipient Physician,39,1003042169,JACQUELINE,Y,LONIER,NaN,JACQUELINE,YUEY,LONIER,...,NaN,NaN,NaN,JACKIE,LONIER,NaN,NaN,NaN,NaN,False
59,Covered Recipient Physician,64,1003081944,TONSLYN,A,TOURE,NaN,TONSLYN,NaN,TOURE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
178,Covered Recipient Physician,196,1003832437,LAWRENCE,FAYEZ,NESHIWAT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
189,Covered Recipient Physician,207,1003839101,THOMAS,M,FLOOD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296428,Covered Recipient Physician/Covered Recipient ...,11042754,1891786422,KIMBERLEE,J,BRONTSEMA,NaN,NaN,NaN,NaN,...,NaN,ROBERT,TOGNACCI,NaN,NaN,NaN,NaN,NaN,NaN,False
1337978,Covered Recipient Physician/Covered Recipient ...,11161690,1992133086,THERESA,J,FRIMEL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1361701,Covered Recipient Physician/Covered Recipient ...,11230513,1245345180,CARLA,J,CULHANE,NaN,CARLA,J,GAROFOLO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1388843,Covered Recipient Physician,11310124,1750623849,LIANG,NaN,XUE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
list_id = list(df_id["covered_recipient_profile_id"])

## Save profile dataset matching with NPI

In [ ]:
npi_specialty.to_csv(path_out_profile + "NPI physician profile data.csv", index=None)
npi_specialty_full.to_csv(path_out_profile + "NPI physician profile data_deactive.csv", index=None)

In [ ]:
df_id.to_csv(path_out_profile + "matched physician profile data.csv", index=None)

#Delete unused dataframe

In [ ]:
del df_profile

# Payment data collection

## General payments

### Extract payment data between 2014 and 2022

#### Define payments data file

In [ ]:
list_general2014 = glob.glob("/content/drive/MyDrive/OPD/2014/OP_DTL_GNRL_PGYR*.csv")
list_general2015 = glob.glob("/content/drive/MyDrive/OPD/2015/OP_DTL_GNRL_PGYR*.csv")
list_general2016 = glob.glob("/content/drive/MyDrive/OPD/2016/OP_DTL_GNRL_PGYR*.csv")
list_general2017 = glob.glob("/content/drive/MyDrive/OPD/2017/OP_DTL_GNRL_PGYR*.csv")
list_general2018 = glob.glob("/content/drive/MyDrive/OPD/2018/OP_DTL_GNRL_PGYR*.csv")
list_general2019 = glob.glob("/content/drive/MyDrive/OPD/2019/OP_DTL_GNRL_PGYR*.csv")
list_general2020 = glob.glob("/content/drive/MyDrive/OPD_g/2020/OP_DTL_GNRL_PGYR*.csv")
list_general2021 = glob.glob("/content/drive/MyDrive/OPD_g/2021/OP_DTL_GNRL_PGYR*.csv")
list_general2022 = glob.glob("/content/drive/MyDrive/OPD_g/2022/OP_DTL_GNRL_PGYR*.csv")

#### 2022 payment data

In [ ]:
df_pay22 = pd.read_csv(list_general2022[0], low_memory=False)
list_df_pay_columns22 = list(df_pay22.columns)
df_pay_id22g = pd.DataFrame(columns=list_df_pay_columns22)

for path_data in list_general2022:
    df_payg = pd.read_csv(path_data, low_memory=False)
    df_payg = df_payg[df_payg["Covered_Recipient_Profile_ID"].isin(list_id)]
    df_payg = df_payg.reset_index(drop=True)
    df_pay_id22g = pd.concat([df_pay_id22g, df_payg], axis=0)
del df_pay22
del list_df_pay_columns22

df_pay_id22g

,Change_Type,Covered_Recipient_Type,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_NPI,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,...,Associated_Drug_or_Biological_NDC_4,Associated_Device_or_Medical_Supply_PDI_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Associated_Device_or_Medical_Supply_PDI_5,Program_Year,Payment_Publication_Date
0,NEW,Covered Recipient Physician,NaN,NaN,NaN,115152.0,1.538107e+09,STEPHEN,L,ARONOFF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,06/30/2023
1,NEW,Covered Recipient Physician,NaN,NaN,NaN,115152.0,1.538107e+09,STEPHEN,L,ARONOFF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,06/30/2023
2,NEW,Covered Recipient Physician,NaN,NaN,NaN,65202.0,1.447337e+09,SHARON,E,SELINGER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,06/30/2023
3,NEW,Covered Recipient Physician,NaN,NaN,NaN,902221.0,1.932369e+09,DANIEL,E,OKORODUDU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,06/30/2023
4,NEW,Covered Recipient Physician,NaN,NaN,NaN,252357.0,1.578518e+09,HEIDI,L,SHEA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,06/30/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21751,NEW,Covered Recipient Physician,NaN,NaN,NaN,1248075.0,1.083846e+09,Jules,NaN,Aljammal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,06/30/2023
21752,NEW,Covered Recipient Physician,NaN,NaN,NaN,219555.0,1.851347e+09,AMMAR,NaN,QOUBAITARY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,06/30/2023
21753,NEW,Covered Recipient Physician,NaN,NaN,NaN,1248075.0,1.083846e+09,Jules,NaN,Aljammal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,06/30/2023
21754,NEW,Covered Recipient Physician,NaN,NaN,NaN,120596.0,1.669459e+09,RICHARD,M,BERGENSTAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,06/30/2023


#### 2014 payment data

In [ ]:
df_pay14 = pd.read_csv(list_general2014[0], low_memory=False)
list_df_pay_columns14 = list(df_pay14.columns)
df_pay_id14g = pd.DataFrame(columns=list_df_pay_columns14)
df_pay_id14g

for path_data in list_general2014:
    df_payg = pd.read_csv(path_data, low_memory=False)
    df_payg = df_payg[df_payg["Physician_Profile_ID"].isin(list_id)]
    df_payg = df_payg.reset_index(drop=True)
    df_pay_id14g = pd.concat([df_pay_id14g, df_payg], axis=0)
del df_pay14

#### 2015 payment data

In [ ]:
df_pay15 = pd.read_csv(list_general2015[0], low_memory=False)
list_df_pay_columns15 = list(df_pay15.columns)
df_pay_id15g = pd.DataFrame(columns=list_df_pay_columns15)
df_pay_id15g

for path_data in list_general2015:
    df_payg = pd.read_csv(path_data, low_memory=False)
    df_payg = df_payg[df_payg["Physician_Profile_ID"].isin(list_id)]
    df_payg = df_payg.reset_index(drop=True)
    df_pay_id15g = pd.concat([df_pay_id15g, df_payg], axis=0)
del df_pay15
del list_df_pay_columns15

#### 2016 payment data

In [ ]:
df_pay16 = pd.read_csv(list_general2016[0], low_memory=False)
list_df_pay_columns16 = list(df_pay16.columns)
df_pay_id16g = pd.DataFrame(columns=list_df_pay_columns16)
df_pay_id16g

for path_data in list_general2016:
    df_payg = pd.read_csv(path_data, low_memory=False)
    df_payg = df_payg[df_payg["Covered_Recipient_Profile_ID"].isin(list_id)]
    df_payg = df_payg.reset_index(drop=True)
    df_pay_id16g = pd.concat([df_pay_id16g, df_payg], axis=0)
del df_pay16
del list_df_pay_columns16

#### 2017 payment data

In [ ]:
df_pay17 = pd.read_csv(list_general2017[0], low_memory=False)
list_df_pay_columns17 = list(df_pay17.columns)
df_pay_id17g = pd.DataFrame(columns=list_df_pay_columns17)
df_pay_id17g

for path_data in list_general2017:
    df_payg = pd.read_csv(path_data, low_memory=False)
    df_payg = df_payg[df_payg["Covered_Recipient_Profile_ID"].isin(list_id)]
    df_payg = df_payg.reset_index(drop=True)
    df_pay_id17g = pd.concat([df_pay_id17g, df_payg], axis=0)
del df_pay17
del list_df_pay_columns17

#### 2018 payment data

In [ ]:
df_pay18 = pd.read_csv(list_general2018[0], low_memory=False)
list_df_pay_columns18 = list(df_pay18.columns)
df_pay_id18g = pd.DataFrame(columns=list_df_pay_columns18)
df_pay_id18g

for path_data in list_general2018:
    df_payg = pd.read_csv(path_data, low_memory=False)
    df_payg = df_payg[df_payg["Covered_Recipient_Profile_ID"].isin(list_id)]
    df_payg = df_payg.reset_index(drop=True)
    df_pay_id18g = pd.concat([df_pay_id18g, df_payg], axis=0)
del df_pay18
del list_df_pay_columns18

#### 2019 payment data

In [ ]:
df_pay19 = pd.read_csv(list_general2019[0], low_memory=False)
list_df_pay_columns19 = list(df_pay19.columns)
df_pay_id19g = pd.DataFrame(columns=list_df_pay_columns19)

for path_data in list_general2019:
    df_payg = pd.read_csv(path_data, low_memory=False)
    df_payg = df_payg[df_payg["Covered_Recipient_Profile_ID"].isin(list_id)]
    df_payg = df_payg.reset_index(drop=True)
    df_pay_id19g = pd.concat([df_pay_id19g, df_payg], axis=0)
del df_pay19
del list_df_pay_columns19

#### 2020 payment data

In [ ]:
df_pay20 = pd.read_csv(list_general2020[0], low_memory=False)
list_df_pay_columns20 = list(df_pay20.columns)
df_pay_id20g = pd.DataFrame(columns=list_df_pay_columns20)

for path_data in list_general2020:
    df_payg = pd.read_csv(path_data, low_memory=False)
    df_payg = df_payg[df_payg["Covered_Recipient_Profile_ID"].isin(list_id)]
    df_payg = df_payg.reset_index(drop=True)
    df_pay_id20g = pd.concat([df_pay_id20g, df_payg], axis=0)
del df_pay20
del list_df_pay_columns20

#### 2021 payment data

In [ ]:
df_pay21 = pd.read_csv(list_general2021[0], low_memory=False)
list_df_pay_columns21 = list(df_pay21.columns)
df_pay_id21g = pd.DataFrame(columns=list_df_pay_columns21)

for path_data in list_general2021:
    df_payg = pd.read_csv(path_data, low_memory=False)
    df_payg = df_payg[df_payg["Covered_Recipient_Profile_ID"].isin(list_id)]
    df_payg = df_payg.reset_index(drop=True)
    df_pay_id21g = pd.concat([df_pay_id21g, df_payg], axis=0)
del df_pay21
del list_df_pay_columns21

df_pay_id21g

,Change_Type,Covered_Recipient_Type,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_NPI,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,...,Associated_Drug_or_Biological_NDC_4,Associated_Device_or_Medical_Supply_PDI_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Associated_Device_or_Medical_Supply_PDI_5,Program_Year,Payment_Publication_Date
0,NEW,Covered Recipient Physician,NaN,NaN,NaN,317531.0,1.003123e+09,GIANLUCA,NaN,IACOBELLIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,06/30/2022
1,NEW,Covered Recipient Physician,NaN,NaN,NaN,1121472.0,1.629162e+09,Elizabeth,NaN,King,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,06/30/2022
2,NEW,Covered Recipient Physician,NaN,NaN,NaN,36160.0,1.801844e+09,Richard,NaN,Sachson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,06/30/2022
3,NEW,Covered Recipient Physician,NaN,NaN,NaN,166619.0,1.689628e+09,Audrey,NaN,Miklius,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,06/30/2022
4,NEW,Covered Recipient Physician,NaN,NaN,NaN,1261902.0,1.598080e+09,Jaime,NaN,Wiebel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,06/30/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,NEW,Covered Recipient Physician,NaN,NaN,NaN,302701.0,1.689656e+09,SONIA,A,TALWAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,06/30/2022
2870,NEW,Covered Recipient Physician,NaN,NaN,NaN,154591.0,1.417161e+09,JENNIFER,C,WHEATON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,06/30/2022
2871,NEW,Covered Recipient Physician,NaN,NaN,NaN,111418.0,1.467481e+09,MEHTAP,NaN,BERKMEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,06/30/2022
2872,NEW,Covered Recipient Physician,NaN,NaN,NaN,111418.0,1.467481e+09,MEHTAP,NaN,BERKMEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,06/30/2022


### Combine each dataset to one dataset

In [ ]:
df1315g = pd.concat([df_pay_id14g, df_pay_id15g], axis = 0)

del df_pay_id14g
del df_pay_id15g

In [ ]:
df1315g= df1315g.rename(columns={"Physician_Profile_ID":"Covered_Recipient_Profile_ID"})
df1315g= df1315g.rename(columns={"Physician_First_Name":"Covered_Recipient_First_Name"})
df1315g= df1315g.rename(columns={"Physician_Middle_Name":"Covered_Recipient_Middle_Name"})
df1315g= df1315g.rename(columns={"Physician_Last_Name":"Covered_Recipient_Last_Name"})
df1315g= df1315g.rename(columns={"Physician_Name_Suffix":"Covered_Recipient_Name_Suffix"})
df1315g= df1315g.rename(columns={"Physician_Primary_Type":"Covered_Recipient_Primary_Type_1"})
df1315g= df1315g.rename(columns={"Physician_Specialty":"Covered_Recipient_Specialty_1"})
df1315g= df1315g.rename(columns={"Physician_License_State_code1" :"Covered_Recipient_License_State_code1"})
df1315g= df1315g.rename(columns={"Physician_License_State_code2" :"Covered_Recipient_License_State_code2"})
df1315g= df1315g.rename(columns={"Physician_License_State_code3" :"Covered_Recipient_License_State_code3"})
df1315g= df1315g.rename(columns={"Physician_License_State_code4" :"Covered_Recipient_License_State_code4"})
df1315g= df1315g.rename(columns={"Physician_License_State_code5" :"Covered_Recipient_License_State_code5"})

In [ ]:
df1315g= df1315g.rename(columns={"Product_Indicator":"Related_Product_Indicator"})
df1315g= df1315g.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological1":"Associated_Drug_or_Biological_NDC_1"})
df1315g= df1315g.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological2":"Associated_Drug_or_Biological_NDC_2"})
df1315g= df1315g.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological3":"Associated_Drug_or_Biological_NDC_3"})
df1315g= df1315g.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological4":"Associated_Drug_or_Biological_NDC_4"})
df1315g= df1315g.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological5":"Associated_Drug_or_Biological_NDC_5"})
df1315g= df1315g.rename(columns = {"Physician_NPI":"Covered_Recipient_NPI"})

In [ ]:
df1315g["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1"] = df1315g['Name_of_Associated_Covered_Drug_or_Biological1'].astype(str)
+ df1315g["Name_of_Associated_Covered_Device_or_Medical_Supply1"]
df1315g["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2"] = df1315g['Name_of_Associated_Covered_Drug_or_Biological2'].astype(str)
+ df1315g["Name_of_Associated_Covered_Device_or_Medical_Supply2"]
df1315g["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3"] = df1315g['Name_of_Associated_Covered_Drug_or_Biological3'].astype(str)
+ df1315g["Name_of_Associated_Covered_Device_or_Medical_Supply3"]
df1315g["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4"] = df1315g['Name_of_Associated_Covered_Drug_or_Biological4'].astype(str)
+ df1315g["Name_of_Associated_Covered_Device_or_Medical_Supply4"]
df1315g["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5"] = df1315g['Name_of_Associated_Covered_Drug_or_Biological5'].astype(str)
+ df1315g["Name_of_Associated_Covered_Device_or_Medical_Supply5"]

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
22646    NaN
22647    NaN
22648    NaN
22649    NaN
22650    NaN
Name: Name_of_Associated_Covered_Device_or_Medical_Supply5, Length: 636650, dtype: object

In [ ]:
df1622g = pd.concat([df_pay_id16g, df_pay_id17g,
                    df_pay_id18g, df_pay_id19g,
                    df_pay_id20g, df_pay_id21g,
                     df_pay_id22g], axis = 0)
del df_pay_id16g
del df_pay_id17g
del df_pay_id18g
del df_pay_id19g
del df_pay_id20g
del df_pay_id21g
del df_pay_id22g
del df_payg

In [ ]:
dfg = pd.concat([df1315g,df1622g], axis = 0)

In [ ]:
del df1315g
del df1622g

In [ ]:
dfg = dfg.drop(["Name_of_Associated_Covered_Device_or_Medical_Supply1",
             "Name_of_Associated_Covered_Device_or_Medical_Supply2",
             "Name_of_Associated_Covered_Device_or_Medical_Supply3",
             "Name_of_Associated_Covered_Device_or_Medical_Supply4",
             "Name_of_Associated_Covered_Device_or_Medical_Supply5",
             "Name_of_Associated_Covered_Drug_or_Biological1",
             "Name_of_Associated_Covered_Drug_or_Biological2",
             "Name_of_Associated_Covered_Drug_or_Biological3",
             "Name_of_Associated_Covered_Drug_or_Biological4",
             "Name_of_Associated_Covered_Drug_or_Biological5"],
             axis=1)
dfg=dfg.replace("nan", "")
dfg=dfg.replace("NaN", "")

In [ ]:
dfg.to_csv(path_out1 + "full_general payments dataset2014-2022.csv", index=None)

In [ ]:
dfg = dfg[['Covered_Recipient_Profile_ID', 'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
           'Change_Type', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
           'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID', 'Total_Amount_of_Payment_USDollars',
           'Date_of_Payment', 'Number_of_Payments_Included_in_Total_Amount',
           'Nature_of_Payment_or_Transfer_of_Value', 'Dispute_Status_for_Publication',
           'Associated_Drug_or_Biological_NDC_1', 'Associated_Drug_or_Biological_NDC_2',
           'Associated_Drug_or_Biological_NDC_3', 'Associated_Drug_or_Biological_NDC_4',
           'Associated_Drug_or_Biological_NDC_5', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
           'Program_Year', 'Contextual_Information',
           'Record_ID']]

In [ ]:
dfg

,Covered_Recipient_Profile_ID,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Change_Type,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Total_Amount_of_Payment_USDollars,Date_of_Payment,Number_of_Payments_Included_in_Total_Amount,Nature_of_Payment_or_Transfer_of_Value,Dispute_Status_for_Publication,...,Associated_Drug_or_Biological_NDC_4,Associated_Drug_or_Biological_NDC_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Program_Year,Contextual_Information,Record_ID
0,282409.0,"ARKRAY USA, Inc.",UNCHANGED,"ARKRAY USA, Inc.",100000061353,40000.00,02/01/2014,3,Consulting Fee,No,...,NaN,NaN,,,,,,2014,NaN,112241264
1,282409.0,"ARKRAY USA, Inc.",UNCHANGED,"ARKRAY USA, Inc.",100000061353,113.03,02/03/2014,3,Food and Beverage,No,...,NaN,NaN,,,,,,2014,NaN,113407608
2,282409.0,"ARKRAY USA, Inc.",UNCHANGED,"ARKRAY USA, Inc.",100000061353,1277.30,05/03/2014,5,Travel and Lodging,No,...,NaN,NaN,,,,,,2014,NaN,113432632
3,809193.0,"LIPOSCIENCE, INC.",UNCHANGED,"LIPOSCIENCE, INC.",100000000116,2000.00,03/01/2014,1,Honoraria,No,...,NaN,NaN,NMR LipoProfile Test,,,,,2014,NaN,110035726
4,26386.0,"LIPOSCIENCE, INC.",UNCHANGED,"LIPOSCIENCE, INC.",100000000116,1000.00,08/01/2014,1,Honoraria,No,...,NaN,NaN,NMR LipoProfile Test,,,,,2014,NaN,110035712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21751,1248075.0,Cambridge Interventional LLC,NEW,Cambridge Interventional LLC,100001176927,1531.25,07/12/2022,3,Consulting Fee,No,...,NaN,NaN,CRF,NaN,NaN,NaN,NaN,2022,NaN,964212219
21752,219555.0,Cambridge Interventional LLC,NEW,Cambridge Interventional LLC,100001176927,1312.50,07/18/2022,1,Education,No,...,NaN,NaN,CRF,NaN,NaN,NaN,NaN,2022,NaN,964164737
21753,1248075.0,Cambridge Interventional LLC,NEW,Cambridge Interventional LLC,100001176927,1333.33,09/19/2022,2,Consulting Fee,No,...,NaN,NaN,CRF,NaN,NaN,NaN,NaN,2022,NaN,964201807
21754,120596.0,"Zealand Pharma US, Inc.",NEW,"Zealand Pharma US, Inc.",100000801822,78.00,01/25/2022,1,Travel and Lodging,No,...,NaN,NaN,ZEGALOGUE,NaN,NaN,NaN,NaN,2022,NaN,905966299


## Direct research payments

###Extract payment data

#### 2014 payment data

In [ ]:
df_pay14r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR14_P012122/OP_DTL_RSRCH_PGYR2014_P01212022.csv",
                        low_memory=False, chunksize= 200000)
df_pay14r = pd.concat((r for r in df_pay14r), ignore_index=True)

In [ ]:
df_pay_id14r = df_pay14r[df_pay14r["Physician_Profile_ID"].isin(list_id)  ]

#### 2015 payment data

In [ ]:
df_pay15r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR15_P063022/OP_DTL_RSRCH_PGYR2015_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay15r = pd.concat((r for r in df_pay15r), ignore_index=True)

In [ ]:
df_pay_id15r = df_pay15r[df_pay15r["Physician_Profile_ID"].isin(list_id) ]

#### 2016 payment data

In [ ]:
df_pay16r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR16_P063022/OP_DTL_RSRCH_PGYR2016_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay16r = pd.concat((r for r in df_pay16r), ignore_index=True)

In [ ]:
df_pay_id16r = df_pay16r[df_pay16r["Covered_Recipient_Profile_ID"].isin(list_id) ]

#### 2017 payment data

In [ ]:
df_pay17r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR17_P063022/OP_DTL_RSRCH_PGYR2017_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay17r = pd.concat((r for r in df_pay17r), ignore_index=True)

In [ ]:
df_pay_id17r = df_pay17r[df_pay17r["Covered_Recipient_Profile_ID"].isin(list_id)  ]

#### 2018 payment data

In [ ]:
df_pay18r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR18_P063022/OP_DTL_RSRCH_PGYR2018_P06302022.csv",
                        low_memory=False, chunksize= 200000)


In [ ]:
df_pay18r = pd.concat((r for r in df_pay18r), ignore_index=True)

In [ ]:
df_pay_id18r = df_pay18r[df_pay18r["Covered_Recipient_Profile_ID"].isin(list_id)  ]

#### 2019 payment data

In [ ]:
df_pay19r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR19_P063022/OP_DTL_RSRCH_PGYR2019_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay19r = pd.concat((r for r in df_pay19r), ignore_index=True)

In [ ]:
df_pay_id19r = df_pay19r[df_pay19r["Covered_Recipient_Profile_ID"].isin(list_id)  ]

#### 2020 payment data

In [ ]:
df_pay20r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR20_P063022/OP_DTL_RSRCH_PGYR2020_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay20r = pd.concat((r for r in df_pay20r), ignore_index=True)

In [ ]:
df_pay_id20r = df_pay20r[df_pay20r["Covered_Recipient_Profile_ID"].isin(list_id) ]

#### 2021 payment data

In [ ]:
df_pay21r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR21_P063022/OP_DTL_RSRCH_PGYR2021_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay21r = pd.concat((r for r in df_pay21r), ignore_index=True)

In [ ]:
df_pay_id21r = df_pay21r[df_pay21r["Covered_Recipient_Profile_ID"].isin(list_id)  ]

#### 2022 payment data

In [ ]:
df_pay22r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR22_P063023/OP_DTL_RSRCH_PGYR2022_P06302023.csv",
                        low_memory=False, chunksize= 200000)
df_pay22r = pd.concat((r for r in df_pay22r), ignore_index=True)
df_pay_id22r = df_pay22r[df_pay22r["Covered_Recipient_Profile_ID"].isin(list_id) ]

### Rename columns

In [ ]:
del df_pay14r
del df_pay15r
del df_pay16r
del df_pay17r
del df_pay18r
del df_pay19r
del df_pay20r
del df_pay21r
del df_pay22r

In [ ]:
df1315r = pd.concat([df_pay_id14r, df_pay_id15r], axis = 0)

In [ ]:
df1315r= df1315r.rename(columns={"Physician_Profile_ID":"Covered_Recipient_Profile_ID"})
df1315r= df1315r.rename(columns={"Physician_First_Name":"Covered_Recipient_First_Name"})
df1315r= df1315r.rename(columns={"Physician_Middle_Name":"Covered_Recipient_Middle_Name"})
df1315r= df1315r.rename(columns={"Physician_Last_Name":"Covered_Recipient_Last_Name"})
df1315r= df1315r.rename(columns={"Physician_Name_Suffix":"Covered_Recipient_Name_Suffix"})
df1315r= df1315r.rename(columns={"Physician_Primary_Type":"Covered_Recipient_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Physician_Specialty":"Covered_Recipient_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_1_Primary_Type":"Principal_Investigator_1_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_1_Specialty":"Principal_Investigator_1_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_2_Primary_Type":"Principal_Investigator_2_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_2_Specialty":"Principal_Investigator_2_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_3_Primary_Type":"Principal_Investigator_3_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_3_Specialty":"Principal_Investigator_3_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_4_Primary_Type":"Principal_Investigator_4_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_4_Specialty":"Principal_Investigator_4_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_5_Primary_Type":"Principal_Investigator_5_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_5_Specialty":"Principal_Investigator_5_Specialty_1"})

In [ ]:
df1315r= df1315r.rename(columns={"Physician_License_State_code1" :"Covered_Recipient_License_State_code1"})
df1315r= df1315r.rename(columns={"Physician_License_State_code2" :"Covered_Recipient_License_State_code2"})
df1315r= df1315r.rename(columns={"Physician_License_State_code3" :"Covered_Recipient_License_State_code3"})
df1315r= df1315r.rename(columns={"Physician_License_State_code4" :"Covered_Recipient_License_State_code4"})
df1315r= df1315r.rename(columns={"Physician_License_State_code5" :"Covered_Recipient_License_State_code5"})

df1315r= df1315r.rename(columns={"Product_Indicator":"Related_Product_Indicator"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological1":"Associated_Drug_or_Biological_NDC_1"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological2":"Associated_Drug_or_Biological_NDC_2"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological3":"Associated_Drug_or_Biological_NDC_3"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological4":"Associated_Drug_or_Biological_NDC_4"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological5":"Associated_Drug_or_Biological_NDC_5"})
df1315r= df1315r.rename(columns = {"Physician_NPI":"Covered_Recipient_Profile_NPI"})

df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological1'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply1"]
df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological2'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply2"]
df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological3'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply3"]
df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological4'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply4"]
df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological5'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply5"]

352       NaN
411       NaN
416       NaN
440       NaN
559       NaN
         ... 
172078    NaN
172141    NaN
172292    NaN
172398    NaN
172411    NaN
Name: Name_of_Associated_Covered_Device_or_Medical_Supply5, Length: 2349, dtype: object

In [ ]:
df1621r = pd.concat([df_pay_id16r, df_pay_id17r,
                    df_pay_id18r, df_pay_id19r,
                    df_pay_id20r, df_pay_id21r,
                     df_pay_id22r], axis = 0)

In [ ]:
dfrd = pd.concat([df1315r, df1621r])

In [ ]:
dfrd = dfrd.drop(["Name_of_Associated_Covered_Device_or_Medical_Supply1",
                  "Name_of_Associated_Covered_Device_or_Medical_Supply2",
                  "Name_of_Associated_Covered_Device_or_Medical_Supply3",
                  "Name_of_Associated_Covered_Device_or_Medical_Supply4",
                  "Name_of_Associated_Covered_Device_or_Medical_Supply5",
                  "Name_of_Associated_Covered_Drug_or_Biological1",
                  "Name_of_Associated_Covered_Drug_or_Biological2",
                  "Name_of_Associated_Covered_Drug_or_Biological3",
                  "Name_of_Associated_Covered_Drug_or_Biological4",
                  "Name_of_Associated_Covered_Drug_or_Biological5"],
             axis=1)
dfrd =dfrd.replace("nan", "")
dfrd =dfrd.replace("NaN", "")


In [ ]:
del df_pay_id14r
del df_pay_id15r
del df_pay_id16r
del df_pay_id17r
del df_pay_id18r
del df_pay_id19r
del df_pay_id20r
del df_pay_id21r
del df_pay_id22r

In [ ]:
del df1315r
del df1621r

In [ ]:
dfrd.to_csv(path_out2 + "full_direct research payments dataset2014-2022.csv", index=None)

In [ ]:
dfrd = dfrd[['Covered_Recipient_Profile_ID',
           'Principal_Investigator_1_Profile_ID', 'Principal_Investigator_2_Profile_ID',
           'Principal_Investigator_3_Profile_ID', 'Principal_Investigator_4_Profile_ID',
           'Principal_Investigator_5_Profile_ID',
           'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
           'Change_Type', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
           'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID', 'Total_Amount_of_Payment_USDollars',
           'Date_of_Payment', 'Preclinical_Research_Indicator',
           'Name_of_Study', 'Dispute_Status_for_Publication',
           'Program_Year',  'ClinicalTrials_Gov_Identifier',
           'Associated_Drug_or_Biological_NDC_1', 'Associated_Drug_or_Biological_NDC_2',
           'Associated_Drug_or_Biological_NDC_3', 'Associated_Drug_or_Biological_NDC_4',
           'Associated_Drug_or_Biological_NDC_5',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
           'Record_ID' ]]

In [ ]:
exclude_d_research = dfrd['Record_ID'].values.tolist()
exclude_d_research

[188267970,
 209452866,
 209452858,
 209452808,
 174396886,
 163698078,
 163680926,
 219308756,
 172269368,
 188226186,
 163770192,
 188234866,
 215886022,
 155694154,
 188264474,
 163782878,
 209452816,
 209452878,
 209452888,
 209452824,
 209452826,
 188264392,
 188264486,
 196509170,
 163696049,
 209453260,
 188262188,
 212922248,
 144089028,
 208636374,
 163772758,
 163706448,
 188226428,
 219308754,
 215886002,
 163751814,
 163759696,
 174363276,
 209453302,
 209452844,
 163694658,
 209452828,
 155694140,
 188264406,
 152805410,
 163721766,
 188268708,
 188268440,
 215885960,
 174396876,
 163705018,
 163688236,
 209453202,
 144065857,
 209453516,
 209452742,
 163698232,
 209452748,
 188264342,
 188264478,
 188234862,
 163694434,
 188272822,
 212902782,
 211286458,
 163770166,
 163681141,
 188226636,
 155694150,
 188264468,
 163731806,
 210218524,
 144068482,
 163693054,
 215885970,
 163688930,
 163690232,
 163697986,
 163761514,
 163706746,
 163689732,
 209452690,
 209453074,
 209

## Associated research payment

###Extract payment data

#### 2014 payment data

In [ ]:
df_pay14r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR14_P012122/OP_DTL_RSRCH_PGYR2014_P01212022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay14r = pd.concat((r for r in df_pay14r), ignore_index=True)

In [ ]:
df_pay_id14r = df_pay14r[df_pay14r["Principal_Investigator_1_Profile_ID"].isin(list_id) |
                      df_pay14r["Principal_Investigator_2_Profile_ID"].isin(list_id) |
                      df_pay14r["Principal_Investigator_3_Profile_ID"].isin(list_id) |
                      df_pay14r["Principal_Investigator_4_Profile_ID"].isin(list_id) |
                      df_pay14r["Principal_Investigator_5_Profile_ID"].isin(list_id) ]

#### 2015 payment data

In [ ]:
df_pay15r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR15_P063022/OP_DTL_RSRCH_PGYR2015_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay15r = pd.concat((r for r in df_pay15r), ignore_index=True)

In [ ]:
df_pay_id15r = df_pay15r[df_pay15r["Principal_Investigator_1_Profile_ID"].isin(list_id) |
                      df_pay15r["Principal_Investigator_2_Profile_ID"].isin(list_id) |
                      df_pay15r["Principal_Investigator_3_Profile_ID"].isin(list_id) |
                      df_pay15r["Principal_Investigator_4_Profile_ID"].isin(list_id) |
                      df_pay15r["Principal_Investigator_5_Profile_ID"].isin(list_id) ]

#### 2016 payment data

In [ ]:
df_pay16r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR16_P063022/OP_DTL_RSRCH_PGYR2016_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay16r = pd.concat((r for r in df_pay16r), ignore_index=True)

In [ ]:
df_pay_id16r = df_pay16r[df_pay16r["Principal_Investigator_1_Profile_ID"].isin(list_id) |
                      df_pay16r["Principal_Investigator_2_Profile_ID"].isin(list_id) |
                      df_pay16r["Principal_Investigator_3_Profile_ID"].isin(list_id) |
                      df_pay16r["Principal_Investigator_4_Profile_ID"].isin(list_id) |
                      df_pay16r["Principal_Investigator_5_Profile_ID"].isin(list_id) ]

#### 2017 payment data

In [ ]:
df_pay17r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR17_P063022/OP_DTL_RSRCH_PGYR2017_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay17r = pd.concat((r for r in df_pay17r), ignore_index=True)

In [ ]:
df_pay_id17r = df_pay17r[df_pay17r["Principal_Investigator_1_Profile_ID"].isin(list_id) |
                      df_pay17r["Principal_Investigator_2_Profile_ID"].isin(list_id) |
                      df_pay17r["Principal_Investigator_3_Profile_ID"].isin(list_id) |
                      df_pay17r["Principal_Investigator_4_Profile_ID"].isin(list_id) |
                      df_pay17r["Principal_Investigator_5_Profile_ID"].isin(list_id) ]

#### 2018 payment data

In [ ]:
df_pay18r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR18_P063022/OP_DTL_RSRCH_PGYR2018_P06302022.csv",
                        low_memory=False, chunksize= 200000)


In [ ]:
df_pay18r = pd.concat((r for r in df_pay18r), ignore_index=True)

In [ ]:
df_pay_id18r = df_pay18r[df_pay18r["Principal_Investigator_1_Profile_ID"].isin(list_id) |
                      df_pay18r["Principal_Investigator_2_Profile_ID"].isin(list_id) |
                      df_pay18r["Principal_Investigator_3_Profile_ID"].isin(list_id) |
                      df_pay18r["Principal_Investigator_4_Profile_ID"].isin(list_id) |
                      df_pay18r["Principal_Investigator_5_Profile_ID"].isin(list_id) ]

#### 2019 payment data

In [ ]:
df_pay19r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR19_P063022/OP_DTL_RSRCH_PGYR2019_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay19r = pd.concat((r for r in df_pay19r), ignore_index=True)

In [ ]:
df_pay_id19r = df_pay19r[df_pay19r["Principal_Investigator_1_Profile_ID"].isin(list_id) |
                      df_pay19r["Principal_Investigator_2_Profile_ID"].isin(list_id) |
                      df_pay19r["Principal_Investigator_3_Profile_ID"].isin(list_id) |
                      df_pay19r["Principal_Investigator_4_Profile_ID"].isin(list_id) |
                      df_pay19r["Principal_Investigator_5_Profile_ID"].isin(list_id) ]

#### 2020 payment data

In [ ]:
df_pay20r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR20_P063022/OP_DTL_RSRCH_PGYR2020_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay20r = pd.concat((r for r in df_pay20r), ignore_index=True)

In [ ]:
df_pay_id20r = df_pay20r[df_pay20r["Principal_Investigator_1_Profile_ID"].isin(list_id) |
                      df_pay20r["Principal_Investigator_2_Profile_ID"].isin(list_id) |
                      df_pay20r["Principal_Investigator_3_Profile_ID"].isin(list_id) |
                      df_pay20r["Principal_Investigator_4_Profile_ID"].isin(list_id) |
                      df_pay20r["Principal_Investigator_5_Profile_ID"].isin(list_id) ]

#### 2021 payment data

In [ ]:
df_pay21r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR21_P063022/OP_DTL_RSRCH_PGYR2021_P06302022.csv",
                        low_memory=False, chunksize= 200000)

In [ ]:
df_pay21r = pd.concat((r for r in df_pay21r), ignore_index=True)

In [ ]:
df_pay_id21r = df_pay21r[df_pay21r["Principal_Investigator_1_Profile_ID"].isin(list_id) |
                      df_pay21r["Principal_Investigator_2_Profile_ID"].isin(list_id) |
                      df_pay21r["Principal_Investigator_3_Profile_ID"].isin(list_id) |
                      df_pay21r["Principal_Investigator_4_Profile_ID"].isin(list_id) |
                      df_pay21r["Principal_Investigator_5_Profile_ID"].isin(list_id) ]

#### 2022 data

In [ ]:
df_pay22r = pd.read_csv("/content/drive/MyDrive/OPD_g/PGYR22_P063023/OP_DTL_RSRCH_PGYR2022_P06302023.csv",
                        low_memory=False, chunksize= 200000)

df_pay22r = pd.concat((r for r in df_pay22r), ignore_index=True)
df_pay_id22r = df_pay22r[df_pay22r["Principal_Investigator_1_Profile_ID"].isin(list_id) |
                      df_pay22r["Principal_Investigator_2_Profile_ID"].isin(list_id) |
                      df_pay22r["Principal_Investigator_3_Profile_ID"].isin(list_id) |
                      df_pay22r["Principal_Investigator_4_Profile_ID"].isin(list_id) |
                      df_pay22r["Principal_Investigator_5_Profile_ID"].isin(list_id) ]

### Rename columns

In [ ]:
del df_pay14r
del df_pay15r
del df_pay16r
del df_pay17r
del df_pay18r
del df_pay19r
del df_pay20r
del df_pay21r
del df_pay22r

In [ ]:
df1315r = pd.concat([df_pay_id14r, df_pay_id15r], axis = 0)

In [ ]:
df1315r= df1315r.rename(columns={"Physician_Profile_ID":"Covered_Recipient_Profile_ID"})
df1315r= df1315r.rename(columns={"Physician_First_Name":"Covered_Recipient_First_Name"})
df1315r= df1315r.rename(columns={"Physician_Middle_Name":"Covered_Recipient_Middle_Name"})
df1315r= df1315r.rename(columns={"Physician_Last_Name":"Covered_Recipient_Last_Name"})
df1315r= df1315r.rename(columns={"Physician_Name_Suffix":"Covered_Recipient_Name_Suffix"})
df1315r= df1315r.rename(columns={"Physician_Primary_Type":"Covered_Recipient_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Physician_Specialty":"Covered_Recipient_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_1_Primary_Type":"Principal_Investigator_1_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_1_Specialty":"Principal_Investigator_1_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_2_Primary_Type":"Principal_Investigator_2_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_2_Specialty":"Principal_Investigator_2_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_3_Primary_Type":"Principal_Investigator_3_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_3_Specialty":"Principal_Investigator_3_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_4_Primary_Type":"Principal_Investigator_4_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_4_Specialty":"Principal_Investigator_4_Specialty_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_5_Primary_Type":"Principal_Investigator_5_Primary_Type_1"})
df1315r= df1315r.rename(columns={"Principal_Investigator_5_Specialty":"Principal_Investigator_5_Specialty_1"})

In [ ]:
df1315r= df1315r.rename(columns={"Physician_License_State_code1" :"Covered_Recipient_License_State_code1"})
df1315r= df1315r.rename(columns={"Physician_License_State_code2" :"Covered_Recipient_License_State_code2"})
df1315r= df1315r.rename(columns={"Physician_License_State_code3" :"Covered_Recipient_License_State_code3"})
df1315r= df1315r.rename(columns={"Physician_License_State_code4" :"Covered_Recipient_License_State_code4"})
df1315r= df1315r.rename(columns={"Physician_License_State_code5" :"Covered_Recipient_License_State_code5"})

df1315r= df1315r.rename(columns={"Product_Indicator":"Related_Product_Indicator"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological1":"Associated_Drug_or_Biological_NDC_1"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological2":"Associated_Drug_or_Biological_NDC_2"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological3":"Associated_Drug_or_Biological_NDC_3"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological4":"Associated_Drug_or_Biological_NDC_4"})
df1315r= df1315r.rename(columns={"NDC_of_Associated_Covered_Drug_or_Biological5":"Associated_Drug_or_Biological_NDC_5"})
df1315r= df1315r.rename(columns = {"Physician_NPI":"Covered_Recipient_Profile_NPI"})

df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological1'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply1"]
df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological2'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply2"]
df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological3'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply3"]
df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological4'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply4"]
df1315r["Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5"] = df1315r['Name_of_Associated_Covered_Drug_or_Biological5'].astype(str)
+ df1315r["Name_of_Associated_Covered_Device_or_Medical_Supply5"]

7         NaN
27        NaN
47        NaN
82        NaN
163       NaN
         ... 
956906    NaN
956916    NaN
956943    NaN
956955    NaN
956957    NaN
Name: Name_of_Associated_Covered_Device_or_Medical_Supply5, Length: 90097, dtype: object

In [ ]:
df1621r = pd.concat([df_pay_id16r, df_pay_id17r,
                     df_pay_id18r, df_pay_id19r,
                     df_pay_id20r, df_pay_id21r,
                     df_pay_id22r], axis = 0)

In [ ]:
dfr = pd.concat([df1315r, df1621r])

In [ ]:
dfr = dfr.drop(["Name_of_Associated_Covered_Device_or_Medical_Supply1",
             "Name_of_Associated_Covered_Device_or_Medical_Supply2",
             "Name_of_Associated_Covered_Device_or_Medical_Supply3",
             "Name_of_Associated_Covered_Device_or_Medical_Supply4",
             "Name_of_Associated_Covered_Device_or_Medical_Supply5",
             "Name_of_Associated_Covered_Drug_or_Biological1",
             "Name_of_Associated_Covered_Drug_or_Biological2",
             "Name_of_Associated_Covered_Drug_or_Biological3",
             "Name_of_Associated_Covered_Drug_or_Biological4",
             "Name_of_Associated_Covered_Drug_or_Biological5"],
             axis=1)
dfr=dfr.replace("nan", "")
dfr=dfr.replace("NaN", "")


In [ ]:
del df_pay_id14r
del df_pay_id15r
del df_pay_id16r
del df_pay_id17r
del df_pay_id18r
del df_pay_id19r
del df_pay_id20r
del df_pay_id21r

In [ ]:
del df1315r
del df1621r

In [ ]:
dfr['PI_count']= dfr[['Principal_Investigator_1_Profile_ID',
           'Principal_Investigator_2_Profile_ID',
           'Principal_Investigator_3_Profile_ID',
           'Principal_Investigator_4_Profile_ID',
           'Principal_Investigator_5_Profile_ID']].count(axis = 1)

dfr['per_payment'] = round((dfr['Total_Amount_of_Payment_USDollars'] / dfr['PI_count']),5)
dfr['per_payment'].replace([np.inf, -np.inf], 0, inplace=True)

dfr['PI1'] = dfr['Principal_Investigator_1_Profile_ID'].isin(list_id)
dfr['PI2'] = dfr['Principal_Investigator_2_Profile_ID'].isin(list_id)
dfr['PI3'] = dfr['Principal_Investigator_3_Profile_ID'].isin(list_id)
dfr['PI4'] = dfr['Principal_Investigator_4_Profile_ID'].isin(list_id)
dfr['PI5'] = dfr['Principal_Investigator_5_Profile_ID'].isin(list_id)

dfr['PI_OPDcount'] = dfr.iloc[:,255:260].sum(axis=1)
dfr['PI_OPD_per_payment'] = dfr['PI_OPDcount'] * dfr['per_payment']

###Exclude associated research payments which overlap with the direct research payments

In [ ]:
dfr2 = dfr
dfr2

,Change_Type,Covered_Recipient_Type,Noncovered_Recipient_Entity_Name,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,...,Associated_Device_or_Medical_Supply_PDI_5,PI_count,per_payment,PI1,PI2,PI3,PI4,PI5,PI_OPDcount,PI_OPD_per_payment
7,UNCHANGED,Covered Recipient Teaching Hospital,NaN,140124.0,1647.0,JOHN H. STROGER JR. HOSP OF COOK CTY,NaN,NaN,NaN,NaN,...,NaN,1,16.35,True,False,False,False,False,1,16.35
27,UNCHANGED,Covered Recipient Teaching Hospital,NaN,10033.0,2026.0,UNIVERSITY OF ALABAMA HOSPITAL,NaN,NaN,NaN,NaN,...,NaN,1,460.00,True,False,False,False,False,1,460.00
47,UNCHANGED,Covered Recipient Teaching Hospital,NaN,420078.0,1905.0,GREENVILLE HOSPITAL CENTER,NaN,NaN,NaN,NaN,...,NaN,1,300.00,True,False,False,False,False,1,300.00
82,UNCHANGED,Covered Recipient Teaching Hospital,NaN,270004.0,2163.0,BILLINGS CLINIC,NaN,NaN,NaN,NaN,...,NaN,1,582.65,True,False,False,False,False,1,582.65
163,UNCHANGED,Covered Recipient Teaching Hospital,NaN,10033.0,2026.0,University Of Alabama Hospital,NaN,NaN,NaN,NaN,...,NaN,1,27335.76,True,False,False,False,False,1,27335.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953234,NEW,Non-covered Recipient Entity,"ENDOCRINE RESEARCH SOLUTIONS, INC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,25.06,True,False,False,False,False,1,25.06
953268,NEW,Non-covered Recipient Entity,UT SOUTHWESTERN CLINICAL TRIALS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,6.50,True,False,False,False,False,1,6.50
953276,NEW,Non-covered Recipient Entity,"LUCAS RESEARCH, INC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,13.50,True,False,False,False,False,1,13.50
953277,NEW,Non-covered Recipient Entity,"MARIN ENDOCRINE CARE AND RESEARCH, INC.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,19.17,True,False,False,False,False,1,19.17


In [ ]:
dfr3 = dfr[dfr['Record_ID'].isin(exclude_d_research)]
dfr3.to_csv(path_out2 + "full_associated research payments dataset2013-2021_exclude.csv", index=None)

In [ ]:
dfr = dfr[~dfr['Record_ID'].isin(exclude_d_research)]
dfr

,Change_Type,Covered_Recipient_Type,Noncovered_Recipient_Entity_Name,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,...,Associated_Device_or_Medical_Supply_PDI_5,PI_count,per_payment,PI1,PI2,PI3,PI4,PI5,PI_OPDcount,PI_OPD_per_payment
7,UNCHANGED,Covered Recipient Teaching Hospital,NaN,140124.0,1647.0,JOHN H. STROGER JR. HOSP OF COOK CTY,NaN,NaN,NaN,NaN,...,NaN,1,16.35,True,False,False,False,False,1,16.35
27,UNCHANGED,Covered Recipient Teaching Hospital,NaN,10033.0,2026.0,UNIVERSITY OF ALABAMA HOSPITAL,NaN,NaN,NaN,NaN,...,NaN,1,460.00,True,False,False,False,False,1,460.00
47,UNCHANGED,Covered Recipient Teaching Hospital,NaN,420078.0,1905.0,GREENVILLE HOSPITAL CENTER,NaN,NaN,NaN,NaN,...,NaN,1,300.00,True,False,False,False,False,1,300.00
82,UNCHANGED,Covered Recipient Teaching Hospital,NaN,270004.0,2163.0,BILLINGS CLINIC,NaN,NaN,NaN,NaN,...,NaN,1,582.65,True,False,False,False,False,1,582.65
163,UNCHANGED,Covered Recipient Teaching Hospital,NaN,10033.0,2026.0,University Of Alabama Hospital,NaN,NaN,NaN,NaN,...,NaN,1,27335.76,True,False,False,False,False,1,27335.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953234,NEW,Non-covered Recipient Entity,"ENDOCRINE RESEARCH SOLUTIONS, INC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,25.06,True,False,False,False,False,1,25.06
953268,NEW,Non-covered Recipient Entity,UT SOUTHWESTERN CLINICAL TRIALS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,6.50,True,False,False,False,False,1,6.50
953276,NEW,Non-covered Recipient Entity,"LUCAS RESEARCH, INC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,13.50,True,False,False,False,False,1,13.50
953277,NEW,Non-covered Recipient Entity,"MARIN ENDOCRINE CARE AND RESEARCH, INC.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,19.17,True,False,False,False,False,1,19.17


In [ ]:
dfr.to_csv(path_out2 + "full_associated research payments dataset2013-2021_revision.csv", index=None)

In [ ]:
dfr2.to_csv(path_out2 + "full_associated research payments dataset2013-2021_initial.csv", index=None)

In [ ]:
dfr = dfr[["Covered_Recipient_Profile_ID",
           'Principal_Investigator_1_Profile_ID', 'Principal_Investigator_2_Profile_ID',
           'Principal_Investigator_3_Profile_ID', 'Principal_Investigator_4_Profile_ID',
           'Principal_Investigator_5_Profile_ID',
           'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
           'Change_Type', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
           'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID', 'Total_Amount_of_Payment_USDollars',
           'Date_of_Payment', 'Preclinical_Research_Indicator',
           'Name_of_Study', 'Dispute_Status_for_Publication',
           'Program_Year',  'ClinicalTrials_Gov_Identifier',
           'Record_ID',
           'Associated_Drug_or_Biological_NDC_1', 'Associated_Drug_or_Biological_NDC_2',
           'Associated_Drug_or_Biological_NDC_3', 'Associated_Drug_or_Biological_NDC_4',
           'Associated_Drug_or_Biological_NDC_5',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
           'PI_OPD_per_payment',
           'per_payment', 'PI_OPDcount'
           ]]

# Adjusting for inflation in US dollars between 2014 and 2022

In [ ]:
# General payments
dfg.loc[dfg["Program_Year"]== 2014, "CPI_payment"] = dfg["Total_Amount_of_Payment_USDollars"] / 0.80892518494473
dfg.loc[dfg["Program_Year"]== 2015, "CPI_payment"] = dfg["Total_Amount_of_Payment_USDollars"] / 0.809885359894757
dfg.loc[dfg["Program_Year"]== 2016, "CPI_payment"] = dfg["Total_Amount_of_Payment_USDollars"] / 0.820102168081871
dfg.loc[dfg["Program_Year"]== 2017, "CPI_payment"] = dfg["Total_Amount_of_Payment_USDollars"] / 0.837573251781108
dfg.loc[dfg["Program_Year"]== 2018, "CPI_payment"] = dfg["Total_Amount_of_Payment_USDollars"] / 0.85803078710427
dfg.loc[dfg["Program_Year"]== 2019, "CPI_payment"] = dfg["Total_Amount_of_Payment_USDollars"] / 0.873578103910748
dfg.loc[dfg["Program_Year"]== 2020, "CPI_payment"] = dfg["Total_Amount_of_Payment_USDollars"] / 0.884355298901437
dfg.loc[dfg["Program_Year"]== 2021, "CPI_payment"] = dfg["Total_Amount_of_Payment_USDollars"] / 0.925902513198135
dfg.loc[dfg["Program_Year"]== 2022, "CPI_payment"] = dfg["Total_Amount_of_Payment_USDollars"] / 1
dfg

,Change_Type,Covered_Recipient_Profile_ID,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Total_Amount_of_Payment_USDollars,Date_of_Payment,Number_of_Payments_Included_in_Total_Amount,Nature_of_Payment_or_Transfer_of_Value,Contextual_Information,...,Associated_Drug_or_Biological_NDC_3,Associated_Drug_or_Biological_NDC_4,Associated_Drug_or_Biological_NDC_5,Program_Year,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,CPI_payment
0,UNCHANGED,282409.0,"ARKRAY USA, Inc.",100000061353,"ARKRAY USA, Inc.",40000.00,02/01/2014,3,Consulting Fee,NaN,...,NaN,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,49448.330630
1,UNCHANGED,282409.0,"ARKRAY USA, Inc.",100000061353,"ARKRAY USA, Inc.",113.03,02/03/2014,3,Food and Beverage,NaN,...,NaN,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,139.728620
2,UNCHANGED,282409.0,"ARKRAY USA, Inc.",100000061353,"ARKRAY USA, Inc.",1277.30,05/03/2014,5,Travel and Lodging,NaN,...,NaN,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,1579.008818
3,UNCHANGED,809193.0,"LIPOSCIENCE, INC.",100000000116,"LIPOSCIENCE, INC.",2000.00,03/01/2014,1,Honoraria,NaN,...,NaN,NaN,NaN,2014,NMR LipoProfile Test,NaN,NaN,NaN,NaN,2472.416531
4,UNCHANGED,26386.0,"LIPOSCIENCE, INC.",100000000116,"LIPOSCIENCE, INC.",1000.00,08/01/2014,1,Honoraria,NaN,...,NaN,NaN,NaN,2014,NMR LipoProfile Test,NaN,NaN,NaN,NaN,1236.208266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400980,NEW,1248075.0,Cambridge Interventional LLC,100001176927,Cambridge Interventional LLC,1531.25,07/12/2022,3,Consulting Fee,NaN,...,NaN,NaN,NaN,2022,CRF,NaN,NaN,NaN,NaN,1531.250000
2400981,NEW,219555.0,Cambridge Interventional LLC,100001176927,Cambridge Interventional LLC,1312.50,07/18/2022,1,Education,NaN,...,NaN,NaN,NaN,2022,CRF,NaN,NaN,NaN,NaN,1312.500000
2400982,NEW,1248075.0,Cambridge Interventional LLC,100001176927,Cambridge Interventional LLC,1333.33,09/19/2022,2,Consulting Fee,NaN,...,NaN,NaN,NaN,2022,CRF,NaN,NaN,NaN,NaN,1333.330000
2400983,NEW,120596.0,"Zealand Pharma US, Inc.",100000801822,"Zealand Pharma US, Inc.",78.00,01/25/2022,1,Travel and Lodging,NaN,...,NaN,NaN,NaN,2022,ZEGALOGUE,NaN,NaN,NaN,NaN,78.000000


In [ ]:
# Associated research payments
dfr.loc[dfr["Program_Year"] ==2014, "CPI_payment"] = dfr["Total_Amount_of_Payment_USDollars"] / 0.80892518494473
dfr.loc[dfr["Program_Year"]==2015, "CPI_payment"] = dfr["Total_Amount_of_Payment_USDollars"] / 0.809885359894757
dfr.loc[dfr["Program_Year"]==2016, "CPI_payment"] = dfr["Total_Amount_of_Payment_USDollars"] / 0.820102168081871
dfr.loc[dfr["Program_Year"]==2017, "CPI_payment"] = dfr["Total_Amount_of_Payment_USDollars"] / 0.837573251781108
dfr.loc[dfr["Program_Year"]==2018, "CPI_payment"] = dfr["Total_Amount_of_Payment_USDollars"] / 0.85803078710427
dfr.loc[dfr["Program_Year"]==2019, "CPI_payment"] = dfr["Total_Amount_of_Payment_USDollars"] / 0.873578103910748
dfr.loc[dfr["Program_Year"]==2020, "CPI_payment"] = dfr["Total_Amount_of_Payment_USDollars"] / 0.884355298901437
dfr.loc[dfr["Program_Year"]==2021, "CPI_payment"] = dfr["Total_Amount_of_Payment_USDollars"] / 0.925902513198135
dfr.loc[dfr["Program_Year"]==2022, "CPI_payment"] = dfr["Total_Amount_of_Payment_USDollars"] / 1

dfr['per_payment'] = round((dfr['CPI_payment'] / dfr['PI_count']),10)
dfr['per_payment'].replace([np.inf, -np.inf], 0, inplace=True)
dfr['PI_OPD_per_payment'] = dfr['PI_OPDcount'] * dfr['per_payment']

dfr

,Change_Type,Covered_Recipient_Type,Noncovered_Recipient_Entity_Name,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,...,PI_count,per_payment,PI1,PI2,PI3,PI4,PI5,PI_OPDcount,PI_OPD_per_payment,CPI_payment
0,UNCHANGED,Covered Recipient Teaching Hospital,NaN,140124.0,1647.0,JOHN H. STROGER JR. HOSP OF COOK CTY,NaN,NaN,NaN,NaN,...,1,20.212005,True,False,False,False,False,1,20.212005,20.212005
1,UNCHANGED,Covered Recipient Teaching Hospital,NaN,10033.0,2026.0,UNIVERSITY OF ALABAMA HOSPITAL,NaN,NaN,NaN,NaN,...,1,568.655802,True,False,False,False,False,1,568.655802,568.655802
2,UNCHANGED,Covered Recipient Teaching Hospital,NaN,420078.0,1905.0,GREENVILLE HOSPITAL CENTER,NaN,NaN,NaN,NaN,...,1,370.862480,True,False,False,False,False,1,370.862480,370.862480
3,UNCHANGED,Covered Recipient Teaching Hospital,NaN,270004.0,2163.0,BILLINGS CLINIC,NaN,NaN,NaN,NaN,...,1,720.276746,True,False,False,False,False,1,720.276746,720.276746
4,UNCHANGED,Covered Recipient Teaching Hospital,NaN,10033.0,2026.0,University Of Alabama Hospital,NaN,NaN,NaN,NaN,...,1,33792.692462,True,False,False,False,False,1,33792.692462,33792.692462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313063,NEW,Non-covered Recipient Entity,"ENDOCRINE RESEARCH SOLUTIONS, INC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,25.060000,True,False,False,False,False,1,25.060000,25.060000
313064,NEW,Non-covered Recipient Entity,UT SOUTHWESTERN CLINICAL TRIALS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,6.500000,True,False,False,False,False,1,6.500000,6.500000
313065,NEW,Non-covered Recipient Entity,"LUCAS RESEARCH, INC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,13.500000,True,False,False,False,False,1,13.500000,13.500000
313066,NEW,Non-covered Recipient Entity,"MARIN ENDOCRINE CARE AND RESEARCH, INC.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,19.170000,True,False,False,False,False,1,19.170000,19.170000


In [ ]:
# Direct research payments
dfrd.loc[dfrd["Program_Year"] ==2014, "CPI_payment"] = dfrd["Total_Amount_of_Payment_USDollars"] / 0.80892518494473
dfrd.loc[dfrd["Program_Year"]==2015, "CPI_payment"] = dfrd["Total_Amount_of_Payment_USDollars"] / 0.809885359894757
dfrd.loc[dfrd["Program_Year"]==2016, "CPI_payment"] = dfrd["Total_Amount_of_Payment_USDollars"] / 0.820102168081871
dfrd.loc[dfrd["Program_Year"]==2017, "CPI_payment"] = dfrd["Total_Amount_of_Payment_USDollars"] / 0.837573251781108
dfrd.loc[dfrd["Program_Year"]==2018, "CPI_payment"] = dfrd["Total_Amount_of_Payment_USDollars"] / 0.85803078710427
dfrd.loc[dfrd["Program_Year"]==2019, "CPI_payment"] = dfrd["Total_Amount_of_Payment_USDollars"] / 0.873578103910748
dfrd.loc[dfrd["Program_Year"]==2020, "CPI_payment"] = dfrd["Total_Amount_of_Payment_USDollars"] / 0.884355298901437
dfrd.loc[dfrd["Program_Year"]==2021, "CPI_payment"] = dfrd["Total_Amount_of_Payment_USDollars"] / 0.925902513198135
dfrd.loc[dfrd["Program_Year"]==2022, "CPI_payment"] = dfrd["Total_Amount_of_Payment_USDollars"] / 1
dfrd

,Change_Type,Covered_Recipient_Type,Noncovered_Recipient_Entity_Name,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,...,Associated_Device_or_Medical_Supply_PDI_3,Covered_or_Noncovered_Indicator_4,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4,Product_Category_or_Therapeutic_Area_4,Associated_Device_or_Medical_Supply_PDI_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Associated_Device_or_Medical_Supply_PDI_5,CPI_payment
0,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,222358.0,OPADA,NaN,ALZOHAILI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741.724959
1,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,168576.0,HOWARD,NaN,HARRISON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.362083
2,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,168576.0,HOWARD,NaN,HARRISON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.362083
3,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,156997.0,ASHWINI,NaN,GORE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.308244
4,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,198071.0,ALEX,NaN,MANZANO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1607.070745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11600,NEW,Covered Recipient Physician,NaN,NaN,NaN,NaN,553080.0,PAOLA,NaN,MANSILLA-LETELIER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.300000
11601,NEW,Covered Recipient Physician,NaN,NaN,NaN,NaN,932767.0,MARTHA,YANETTY,GOMEZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15961.000000
11602,NEW,Covered Recipient Physician,NaN,NaN,NaN,NaN,263633.0,GEORGE,L,BAKRIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7063.610000
11603,NEW,Covered Recipient Physician,NaN,NaN,NaN,NaN,333964.0,ALTAGRACIA,A,ALCANTARA GONZALEZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,520.000000


# Checking payments

## General payments

### Checking general payments

In [ ]:
dfg.sort_values(by=['CPI_payment'], ascending=False)

,Change_Type,Covered_Recipient_Profile_ID,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Total_Amount_of_Payment_USDollars,Date_of_Payment,Number_of_Payments_Included_in_Total_Amount,Nature_of_Payment_or_Transfer_of_Value,Contextual_Information,...,Associated_Drug_or_Biological_NDC_3,Associated_Drug_or_Biological_NDC_4,Associated_Drug_or_Biological_NDC_5,Program_Year,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,CPI_payment
614135,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1826075.0,11/16/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.254733e+06
614136,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1734921.0,05/14/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.142181e+06
907670,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1720480.0,08/14/2016,1,Royalty or License,NaN,...,NaN,NaN,NaN,2016,BYETTA,NaN,NaN,NaN,NaN,2.097885e+06
614134,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1695981.0,08/20/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.094100e+06
907669,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1653615.0,11/02/2016,1,Royalty or License,NaN,...,NaN,NaN,NaN,2016,BYETTA,NaN,NaN,NaN,NaN,2.016352e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135805,UNCHANGED,54579.0,Sanofi and Genzyme US Companies,100000000076,SANOFI-AVENTIS U.S. LLC,0.0,11/21/2014,1,Compensation for services other than consultin...,NaN,...,NaN,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,0.000000e+00
135817,UNCHANGED,279588.0,Sanofi and Genzyme US Companies,100000000076,SANOFI-AVENTIS U.S. LLC,0.0,10/01/2014,1,Compensation for services other than consultin...,NaN,...,NaN,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,0.000000e+00
138548,UNCHANGED,54579.0,Sanofi and Genzyme US Companies,100000000076,SANOFI-AVENTIS U.S. LLC,0.0,11/21/2014,1,Travel and Lodging,NaN,...,NaN,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,0.000000e+00
136021,UNCHANGED,825707.0,Sanofi and Genzyme US Companies,100000000076,SANOFI-AVENTIS U.S. LLC,0.0,05/15/2014,1,Compensation for services other than consultin...,NaN,...,NaN,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,0.000000e+00


In [ ]:
dfg_check1 = dfg[dfg['Total_Amount_of_Payment_USDollars'] > 1000000]
dfg['Total_Amount_of_Payment_USDollars'][dfg['Total_Amount_of_Payment_USDollars'] > 1000000].count()

16

In [ ]:
dfg_check1

,Change_Type,Covered_Recipient_Profile_ID,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Total_Amount_of_Payment_USDollars,Date_of_Payment,Number_of_Payments_Included_in_Total_Amount,Nature_of_Payment_or_Transfer_of_Value,Contextual_Information,...,Associated_Drug_or_Biological_NDC_3,Associated_Drug_or_Biological_NDC_4,Associated_Drug_or_Biological_NDC_5,Program_Year,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,CPI_payment
300650,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1485149.0,05/15/2014,1,Royalty or License,NaN,...,NaN,NaN,NaN,2014,BYETTA,NaN,NaN,NaN,NaN,1.835953e+06
300651,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1477915.0,08/15/2014,1,Royalty or License,NaN,...,NaN,NaN,NaN,2014,BYETTA,NaN,NaN,NaN,NaN,1.827011e+06
300652,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1627835.0,11/13/2014,1,Royalty or License,NaN,...,NaN,NaN,NaN,2014,BYETTA,NaN,NaN,NaN,NaN,2.012343e+06
614133,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1407050.0,02/15/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,1.737345e+06
614134,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1695981.0,08/20/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.094100e+06
614135,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1826075.0,11/16/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.254733e+06
614136,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1734921.0,05/14/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.142181e+06
907669,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1653615.0,11/02/2016,1,Royalty or License,NaN,...,NaN,NaN,NaN,2016,BYETTA,NaN,NaN,NaN,NaN,2.016352e+06
907670,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1720480.0,08/14/2016,1,Royalty or License,NaN,...,NaN,NaN,NaN,2016,BYETTA,NaN,NaN,NaN,NaN,2.097885e+06
907671,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1505772.0,05/15/2016,1,Royalty or License,NaN,...,NaN,NaN,NaN,2016,BYETTA,NaN,NaN,NaN,NaN,1.836079e+06


### Replacing categories of several general payments

In [ ]:
dfg_check2 = dfg[dfg['Total_Amount_of_Payment_USDollars'] > 100000]

dfg_check2.to_csv(path_out_profile + "generalpayments_over_USD100000.csv", index=None)

,Change_Type,Covered_Recipient_Profile_ID,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Total_Amount_of_Payment_USDollars,Date_of_Payment,Number_of_Payments_Included_in_Total_Amount,Nature_of_Payment_or_Transfer_of_Value,Contextual_Information,...,Associated_Drug_or_Biological_NDC_3,Associated_Drug_or_Biological_NDC_4,Associated_Drug_or_Biological_NDC_5,Program_Year,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,CPI_payment
128849,UNCHANGED,64469.0,"Dexcom, Inc.",100000010478,"Dexcom, Inc.",232498.00,05/30/2014,1,Current or prospective ownership or investment...,Annual stock grant to member of Dexcom Board o...,...,NaN,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,2.874159e+05
300650,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1485149.00,05/15/2014,1,Royalty or License,NaN,...,NaN,NaN,NaN,2014,BYETTA,NaN,NaN,NaN,NaN,1.835953e+06
300651,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1477915.00,08/15/2014,1,Royalty or License,NaN,...,NaN,NaN,NaN,2014,BYETTA,NaN,NaN,NaN,NaN,1.827011e+06
300652,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1627835.00,11/13/2014,1,Royalty or License,NaN,...,NaN,NaN,NaN,2014,BYETTA,NaN,NaN,NaN,NaN,2.012343e+06
570905,UNCHANGED,64469.0,"Dexcom, Inc.",100000010478,"Dexcom, Inc.",324344.00,06/03/2015,1,Consulting Fee,BOARD OF DIRECTORS - MEMBER COMPENSATION,...,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,4.004814e+05
614133,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1407050.00,02/15/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,1.737345e+06
614134,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1695981.00,08/20/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.094100e+06
614135,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1826075.00,11/16/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.254733e+06
614136,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1734921.00,05/14/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.142181e+06
803399,UNCHANGED,64469.0,"Dexcom, Inc.",100000010478,"Dexcom, Inc.",327318.52,05/25/2016,1,Consulting Fee,Member of Dexcom's Board of Directors,...,NaN,NaN,NaN,2016,Dexcom G5 Mobile CGM System,NaN,NaN,NaN,NaN,3.991192e+05


In [ ]:
dfg.loc[dfg['Record_ID'].isin([335003640, 391437106, 438214637, 605805005, 703398615]),
              'Nature_of_Payment_or_Transfer_of_Value'] = 'Current or prospective ownership or investment interest'

dfg_check3 = dfg[dfg['Total_Amount_of_Payment_USDollars'] > 100000]

,Change_Type,Covered_Recipient_Profile_ID,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Total_Amount_of_Payment_USDollars,Date_of_Payment,Number_of_Payments_Included_in_Total_Amount,Nature_of_Payment_or_Transfer_of_Value,Contextual_Information,...,Associated_Drug_or_Biological_NDC_3,Associated_Drug_or_Biological_NDC_4,Associated_Drug_or_Biological_NDC_5,Program_Year,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,CPI_payment
128849,UNCHANGED,64469.0,"Dexcom, Inc.",100000010478,"Dexcom, Inc.",232498.00,05/30/2014,1,Current or prospective ownership or investment...,Annual stock grant to member of Dexcom Board o...,...,NaN,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,2.874159e+05
300650,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1485149.00,05/15/2014,1,Royalty or License,NaN,...,NaN,NaN,NaN,2014,BYETTA,NaN,NaN,NaN,NaN,1.835953e+06
300651,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1477915.00,08/15/2014,1,Royalty or License,NaN,...,NaN,NaN,NaN,2014,BYETTA,NaN,NaN,NaN,NaN,1.827011e+06
300652,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1627835.00,11/13/2014,1,Royalty or License,NaN,...,NaN,NaN,NaN,2014,BYETTA,NaN,NaN,NaN,NaN,2.012343e+06
570905,UNCHANGED,64469.0,"Dexcom, Inc.",100000010478,"Dexcom, Inc.",324344.00,06/03/2015,1,Current or prospective ownership or investment...,BOARD OF DIRECTORS - MEMBER COMPENSATION,...,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,4.004814e+05
614133,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1407050.00,02/15/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,1.737345e+06
614134,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1695981.00,08/20/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.094100e+06
614135,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1826075.00,11/16/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.254733e+06
614136,UNCHANGED,663788.0,AstraZeneca Pharmaceuticals LP,100000000146,AstraZeneca Pharmaceuticals LP,1734921.00,05/14/2015,1,Royalty or License,NaN,...,NaN,NaN,NaN,2015,BYETTA,NaN,NaN,NaN,NaN,2.142181e+06
803399,UNCHANGED,64469.0,"Dexcom, Inc.",100000010478,"Dexcom, Inc.",327318.52,05/25/2016,1,Current or prospective ownership or investment...,Member of Dexcom's Board of Directors,...,NaN,NaN,NaN,2016,Dexcom G5 Mobile CGM System,NaN,NaN,NaN,NaN,3.991192e+05


### General payments for debt forgiveness

In [ ]:
dfg_check_debt = dfg[dfg['Nature_of_Payment_or_Transfer_of_Value'] == "Debt forgiveness"]
dfg_check_debt

,Change_Type,Covered_Recipient_Profile_ID,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Total_Amount_of_Payment_USDollars,Date_of_Payment,Number_of_Payments_Included_in_Total_Amount,Nature_of_Payment_or_Transfer_of_Value,Contextual_Information,...,Associated_Drug_or_Biological_NDC_3,Associated_Drug_or_Biological_NDC_4,Associated_Drug_or_Biological_NDC_5,Program_Year,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,CPI_payment
2204819,NEW,331472.0,Welch Allyn,100000010393,Welch Allyn,12.62,04/01/2021,1,Debt forgiveness,Open Account Receivable Write Off,...,NaN,NaN,NaN,2021,None,NaN,NaN,NaN,NaN,13.629945
2211203,NEW,513679.0,Baxter Healthcare,100000011010,Baxter Healthcare,12.29,01/06/2022,1,Debt forgiveness,NaN,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,12.290000
2211204,NEW,673535.0,Baxter Healthcare,100000011010,Baxter Healthcare,12.29,01/06/2022,1,Debt forgiveness,NaN,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,12.290000
2211205,NEW,60940.0,Baxter Healthcare,100000011010,Baxter Healthcare,628.00,01/06/2022,1,Debt forgiveness,NaN,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,628.000000
2211206,NEW,127179.0,Baxter Healthcare,100000011010,Baxter Healthcare,278.72,01/06/2022,1,Debt forgiveness,NaN,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,278.720000
2211207,NEW,119966.0,Baxter Healthcare,100000011010,Baxter Healthcare,298.25,03/03/2022,1,Debt forgiveness,NaN,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,298.250000
2211208,NEW,77352.0,Baxter Healthcare,100000011010,Baxter Healthcare,298.25,03/03/2022,1,Debt forgiveness,NaN,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,298.250000
2211209,NEW,221276.0,Baxter Healthcare,100000011010,Baxter Healthcare,298.25,03/03/2022,1,Debt forgiveness,NaN,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,298.250000
2211210,NEW,102842.0,Baxter Healthcare,100000011010,Baxter Healthcare,298.25,03/03/2022,1,Debt forgiveness,NaN,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,298.250000
2211211,NEW,212329.0,Baxter Healthcare,100000011010,Baxter Healthcare,1398.72,01/25/2022,1,Debt forgiveness,NaN,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,1398.720000


### Number of general payments and monetary amounts of general payments

In [ ]:
dfg["Number_of_Payments_Included_in_Total_Amount"].sum()

2474513

In [ ]:
dfg.groupby(['Dispute_Status_for_Publication'])["Number_of_Payments_Included_in_Total_Amount"].sum()

Dispute_Status_for_Publication
No     2474328
Yes        185
Name: Number_of_Payments_Included_in_Total_Amount, dtype: int64

In [ ]:
dfg.groupby(['Dispute_Status_for_Publication', 'Program_Year'])["Number_of_Payments_Included_in_Total_Amount"].sum()

Dispute_Status_for_Publication  Program_Year
No                              2014            309338
                                2015            344281
                                2016            346771
                                2017            337210
                                2018            306492
                                2019            285472
                                2020            161736
                                2021            184884
                                2022            198144
Yes                             2014                77
                                2015                 6
                                2016                 5
                                2017                18
                                2018                28
                                2019                17
                                2020                25
                                2021                 1
                    

In [ ]:
dfg.groupby(['Dispute_Status_for_Publication', 'Change_Type'])["Number_of_Payments_Included_in_Total_Amount"].sum()

Dispute_Status_for_Publication  Change_Type
No                              ADD                921
                                CHANGED           1173
                                NEW             384036
                                UNCHANGED      2088198
Yes                             NEW                  9
                                UNCHANGED          176
Name: Number_of_Payments_Included_in_Total_Amount, dtype: int64

In [ ]:
dfg.groupby(['Contextual_Information'])["Number_of_Payments_Included_in_Total_Amount"].sum()

Contextual_Information
.                                                                      1
02953.01-HF-071716                                                     1
1 Consulting Fee                                                       2
1 hour product feedback call with marketing for next gen. dxa unit     1
1 meal/break                                                          97
                                                                      ..
food and beverage for all day meeting                                 22
inservice                                                              1
loaner product provided for more than 90 days                          1
referral lunch Dr Walsh Dr guleria Palm Springs                        1
review of mens health portfolio                                        1
Name: Number_of_Payments_Included_in_Total_Amount, Length: 17141, dtype: int64

In [ ]:
dfg['CPI_payment'].sum()

649638179.6661197

In [ ]:
dfg.groupby('Program_Year')['CPI_payment'].sum()

Program_Year
2014    9.495666e+07
2015    1.006486e+08
2016    9.346167e+07
2017    1.024573e+08
2018    8.169031e+07
2019    6.895436e+07
2020    3.586397e+07
2021    3.438826e+07
2022    3.721708e+07
Name: CPI_payment, dtype: float64

In [ ]:
dfg.groupby('Program_Year')['Number_of_Payments_Included_in_Total_Amount'].sum()

Program_Year
2014    309415
2015    344287
2016    346776
2017    337228
2018    306520
2019    285489
2020    161761
2021    184885
2022    198152
Name: Number_of_Payments_Included_in_Total_Amount, dtype: int64

In [ ]:
dfg[(dfg['Nature_of_Payment_or_Transfer_of_Value'] != "Current or prospective ownership or investment interest") &
    (dfg['Nature_of_Payment_or_Transfer_of_Value'] != "Royalty or License") &
    (dfg['Nature_of_Payment_or_Transfer_of_Value'] != "Long term medical supply or device loan") &
    (dfg['Nature_of_Payment_or_Transfer_of_Value'] != "Debt forgiveness") &
    (dfg['Nature_of_Payment_or_Transfer_of_Value'] != "Acquisition")].groupby('Program_Year')['Number_of_Payments_Included_in_Total_Amount'].sum()

Program_Year
2014    309406
2015    344276
2016    346762
2017    337222
2018    306517
2019    285485
2020    161753
2021    184880
2022    198145
Name: Number_of_Payments_Included_in_Total_Amount, dtype: int64

## Associated research payments

In [ ]:
round(dfr.agg({'PI_OPD_per_payment': ['sum','count']}),2)

,PI_OPD_per_payment
sum,1.378869e+09
count,3.130680e+05


In [ ]:
round(dfr['PI_OPD_per_payment'].sum(), 2)

1378868830.06

In [ ]:
dfr

,Change_Type,Covered_Recipient_Type,Noncovered_Recipient_Entity_Name,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,...,PI_count,per_payment,PI1,PI2,PI3,PI4,PI5,PI_OPDcount,PI_OPD_per_payment,CPI_payment
0,UNCHANGED,Covered Recipient Teaching Hospital,NaN,140124.0,1647.0,JOHN H. STROGER JR. HOSP OF COOK CTY,NaN,NaN,NaN,NaN,...,1,20.212005,True,False,False,False,False,1,20.212005,20.212005
1,UNCHANGED,Covered Recipient Teaching Hospital,NaN,10033.0,2026.0,UNIVERSITY OF ALABAMA HOSPITAL,NaN,NaN,NaN,NaN,...,1,568.655802,True,False,False,False,False,1,568.655802,568.655802
2,UNCHANGED,Covered Recipient Teaching Hospital,NaN,420078.0,1905.0,GREENVILLE HOSPITAL CENTER,NaN,NaN,NaN,NaN,...,1,370.862480,True,False,False,False,False,1,370.862480,370.862480
3,UNCHANGED,Covered Recipient Teaching Hospital,NaN,270004.0,2163.0,BILLINGS CLINIC,NaN,NaN,NaN,NaN,...,1,720.276746,True,False,False,False,False,1,720.276746,720.276746
4,UNCHANGED,Covered Recipient Teaching Hospital,NaN,10033.0,2026.0,University Of Alabama Hospital,NaN,NaN,NaN,NaN,...,1,33792.692462,True,False,False,False,False,1,33792.692462,33792.692462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313063,NEW,Non-covered Recipient Entity,"ENDOCRINE RESEARCH SOLUTIONS, INC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,25.060000,True,False,False,False,False,1,25.060000,25.060000
313064,NEW,Non-covered Recipient Entity,UT SOUTHWESTERN CLINICAL TRIALS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,6.500000,True,False,False,False,False,1,6.500000,6.500000
313065,NEW,Non-covered Recipient Entity,"LUCAS RESEARCH, INC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,13.500000,True,False,False,False,False,1,13.500000,13.500000
313066,NEW,Non-covered Recipient Entity,"MARIN ENDOCRINE CARE AND RESEARCH, INC.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,19.170000,True,False,False,False,False,1,19.170000,19.170000


In [ ]:
round(dfr['PI_OPD_per_payment'].sum(), 2)

1378868830.06

In [ ]:
round(dfr['PI_OPD_per_payment'].count(),0)

313068

In [ ]:
round(dfr.groupby('Program_Year')['PI_OPD_per_payment'].count(), 2)

Program_Year
2014    41926
2015    48144
2016    38142
2017    31517
2018    27679
2019    19071
2020    30708
2021    26234
2022    49647
Name: PI_OPD_per_payment, dtype: int64

In [ ]:
round(dfr.groupby('Preclinical_Research_Indicator').agg({'CPI_payment': ['sum','count']}),2)

CPI_payment        
                                         sum   count
Preclinical_Research_Indicator                      
No                              1.348721e+09  312595
Yes                             5.475976e+07     473

In [ ]:
dfr.groupby('Covered_Recipient_Type').agg({'PI_OPD_per_payment': ['sum','count']})

PI_OPD_per_payment        
                                                   sum   count
Covered_Recipient_Type                                        
Covered Recipient Physician               1.270465e+05      42
Covered Recipient Teaching Hospital       1.439986e+08   23869
Non-covered Recipient Entity              1.234736e+09  289137
Non-covered Recipient Individual          6.847093e+03      20

In [ ]:
round(dfr[dfr['Covered_Recipient_Type'] == "Covered Recipient Teaching Hospital"]['PI_OPD_per_payment'].sum(), 2)

143998594.96

In [ ]:
round(dfr[dfr['Covered_Recipient_Type'] ==
      "Non-covered Recipient Entity"]['PI_OPD_per_payment'].sum(), 2)

1234736341.46

In [ ]:
round(dfr[dfr['Covered_Recipient_Type'] ==
      "Covered Recipient Physician"]['PI_OPD_per_payment'].sum(), 2)

127046.55

In [ ]:
round(dfr[dfr['Covered_Recipient_Type'] ==
      "Non-covered Recipient Individual"]['PI_OPD_per_payment'].sum(), 2)

6847.09

In [ ]:
dfr["CPI_payment"].count()

313068

In [ ]:
dfr.groupby(['Dispute_Status_for_Publication'])["CPI_payment"].count()

Dispute_Status_for_Publication
No     312813
Yes       255
Name: CPI_payment, dtype: int64

In [ ]:
dfr.groupby(['Dispute_Status_for_Publication', 'Change_Type'])["PI_OPD_per_payment"].count()

Dispute_Status_for_Publication  Change_Type
No                              ADD             15985
                                CHANGED            17
                                NEW             77692
                                UNCHANGED      219119
Yes                             NEW                 1
                                UNCHANGED         254
Name: PI_OPD_per_payment, dtype: int64

## Direct research payments

In [ ]:
dfrd["CPI_payment"].count()

11605

In [ ]:
dfrd.groupby(['Dispute_Status_for_Publication', 'Change_Type'])["CPI_payment"].count()

Dispute_Status_for_Publication  Change_Type
No                              ADD             434
                                NEW            1765
                                UNCHANGED      9406
Name: CPI_payment, dtype: int64

In [ ]:
dfrd['CPI_payment'].count()

11605

In [ ]:
dfrd.groupby('Program_Year')['CPI_payment'].count()

Program_Year
2014    1168
2015    1181
2016    1681
2017    1647
2018    1476
2019    1186
2020    1681
2021     776
2022     809
Name: CPI_payment, dtype: int64

# Pivot table

## Make pivot tables by physician profile ID

### General payments pivot by physician profile ID

In [ ]:
# Including ownership and royalties
df_general_pivot = dfg.pivot_table(index ='Covered_Recipient_Profile_ID',
                                   columns = 'Program_Year',
                                  values= 'CPI_payment',
                                   aggfunc = ['sum'] ).fillna(0)
df_general_pivot['id', 'covered_recipient_profile_id'] = df_general_pivot.index

df_general_pivot = df_general_pivot.reset_index(level=0, drop=True)
df_general_pivot.columns = df_general_pivot.columns.droplevel(0)
df_general_pivot = df_general_pivot.reset_index()
df_general_pivot.drop('index', axis=1, inplace=True)

df_general_pivot.rename(columns = {2014:'general2014',
                                   2015:'general2015',
                                   2016:'general2016',
                                   2017:'general2017',
                                   2018:'general2018',
                                   2019:'general2019',
                                   2020:'general2020',
                                   2021:'general2021',
                                   2022: 'general2022'}
                        , inplace = True)

df_general_pivot.index = df_general_pivot['covered_recipient_profile_id']
df_general_pivot.drop('covered_recipient_profile_id', axis=1, inplace=True)

df_general_pivot["index"] = df_general_pivot.index
df_general_pivot

Program_Year,general2014,general2015,general2016,general2017,general2018,general2019,general2020,general2021,general2022,index
covered_recipient_profile_id,,,,,,,,,,
4.0,95.657796,364.557769,323.727958,89.866767,184.935089,0.000000,13.897129,29.020334,11.76,4.0
39.0,5382.920548,58.428022,27.350251,0.000000,0.000000,160.512265,0.000000,0.000000,0.00,39.0
64.0,548.394349,1540.773623,942.138712,44.951292,0.000000,0.000000,0.000000,0.000000,0.00,64.0
196.0,4196.729269,4962.183784,6031.760667,5222.468591,4987.152051,5914.628557,1159.703573,2652.482270,2696.82,196.0
207.0,32217.342821,39515.358080,6775.180235,1118.684244,18.600731,164.358515,0.000000,257.597314,50.01,207.0
...,...,...,...,...,...,...,...,...,...,...
11042754.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,596.250677,1508.83,11042754.0
11161690.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28.269181,2329.629706,499.07,11161690.0
11230513.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,152.208249,86.50,11230513.0


#### Combine pivot table with the NPI profile dataset

In [ ]:
df_id.index = df_id['covered_recipient_profile_id']
df_id2 = pd.concat([df_id[['covered_recipient_profile_type',
                           'covered_recipient_npi',
                           'covered_recipient_profile_primary_specialty',
                           'has_multiple_ids']], df_general_pivot],
                        axis = 1)
df_id2

,covered_recipient_profile_type,covered_recipient_npi,covered_recipient_profile_primary_specialty,has_multiple_ids,general2014,general2015,general2016,general2017,general2018,general2019,general2020,general2021,general2022,index
covered_recipient_profile_id,,,,,,,,,,,,,,
4.0,Covered Recipient Physician,1003024811,Allopathic & Osteopathic Physicians|Internal M...,False,95.657796,364.557769,323.727958,89.866767,184.935089,0.000000,13.897129,29.020334,11.76,4.0
39.0,Covered Recipient Physician,1003042169,Allopathic & Osteopathic Physicians|Internal M...,False,5382.920548,58.428022,27.350251,0.000000,0.000000,160.512265,0.000000,0.000000,0.00,39.0
64.0,Covered Recipient Physician,1003081944,Allopathic & Osteopathic Physicians|Internal M...,False,548.394349,1540.773623,942.138712,44.951292,0.000000,0.000000,0.000000,0.000000,0.00,64.0
196.0,Covered Recipient Physician,1003832437,Allopathic & Osteopathic Physicians|Internal M...,False,4196.729269,4962.183784,6031.760667,5222.468591,4987.152051,5914.628557,1159.703573,2652.482270,2696.82,196.0
207.0,Covered Recipient Physician,1003839101,Allopathic & Osteopathic Physicians|Internal M...,False,32217.342821,39515.358080,6775.180235,1118.684244,18.600731,164.358515,0.000000,257.597314,50.01,207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11042754.0,Covered Recipient Physician/Covered Recipient ...,1891786422,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,596.250677,1508.83,11042754.0
11161690.0,Covered Recipient Physician/Covered Recipient ...,1992133086,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28.269181,2329.629706,499.07,11161690.0
11230513.0,Covered Recipient Physician/Covered Recipient ...,1245345180,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,152.208249,86.50,11230513.0


In [ ]:
df_general_pivot = dfg.pivot_table(index ='Covered_Recipient_Profile_ID',
                                   columns = 'Program_Year',
                                  values= 'Number_of_Payments_Included_in_Total_Amount',
                                   aggfunc = ['sum'] ).fillna(0)
df_general_pivot['id', 'covered_recipient_profile_id'] = df_general_pivot.index

df_general_pivot = df_general_pivot.reset_index(level=0, drop=True)
df_general_pivot.columns = df_general_pivot.columns.droplevel(0)
df_general_pivot = df_general_pivot.reset_index()
df_general_pivot.drop('index', axis=1, inplace=True)

df_general_pivot.rename(columns = {2014:'generalc2014',
                                   2015:'generalc2015',
                                   2016:'generalc2016',
                                   2017:'generalc2017',
                                   2018:'generalc2018',
                                   2019:'generalc2019',
                                   2020:'generalc2020',
                                   2021:'generalc2021',
                                   2022: 'generalc2022'}
                        , inplace = True)

df_general_pivot.index = df_general_pivot['covered_recipient_profile_id']
df_general_pivot.drop('covered_recipient_profile_id', axis=1, inplace=True)


df_general_pivot["index"] = df_general_pivot.index

df_id2 = pd.concat([df_id2, df_general_pivot],
                        axis = 1)
df_id2

,covered_recipient_profile_type,covered_recipient_npi,covered_recipient_profile_primary_specialty,has_multiple_ids,general2014,general2015,general2016,general2017,general2018,general2019,...,generalc2014,generalc2015,generalc2016,generalc2017,generalc2018,generalc2019,generalc2020,generalc2021,generalc2022,index
covered_recipient_profile_id,,,,,,,,,,,,,,,,,,,,,
4.0,Covered Recipient Physician,1003024811,Allopathic & Osteopathic Physicians|Internal M...,False,95.657796,364.557769,323.727958,89.866767,184.935089,0.000000,...,6.0,8.0,6.0,1.0,6.0,0.0,1.0,2.0,1.0,4.0
39.0,Covered Recipient Physician,1003042169,Allopathic & Osteopathic Physicians|Internal M...,False,5382.920548,58.428022,27.350251,0.000000,0.000000,160.512265,...,5.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,39.0
64.0,Covered Recipient Physician,1003081944,Allopathic & Osteopathic Physicians|Internal M...,False,548.394349,1540.773623,942.138712,44.951292,0.000000,0.000000,...,29.0,61.0,26.0,1.0,0.0,0.0,0.0,0.0,0.0,64.0
196.0,Covered Recipient Physician,1003832437,Allopathic & Osteopathic Physicians|Internal M...,False,4196.729269,4962.183784,6031.760667,5222.468591,4987.152051,5914.628557,...,174.0,166.0,239.0,244.0,232.0,187.0,53.0,101.0,119.0,196.0
207.0,Covered Recipient Physician,1003839101,Allopathic & Osteopathic Physicians|Internal M...,False,32217.342821,39515.358080,6775.180235,1118.684244,18.600731,164.358515,...,60.0,46.0,32.0,20.0,1.0,3.0,0.0,2.0,1.0,207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11042754.0,Covered Recipient Physician/Covered Recipient ...,1891786422,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,37.0,11042754.0
11161690.0,Covered Recipient Physician/Covered Recipient ...,1992133086,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,135.0,23.0,11161690.0
11230513.0,Covered Recipient Physician/Covered Recipient ...,1245345180,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,11230513.0


### By payment categories

In [ ]:
dfg["month"] = dfg["Date_of_Payment"].str[:2].astype(int)
dfg["day"] = dfg["Date_of_Payment"].str.slice(start=3, stop=5).astype(int)
dfg["year"] = dfg["Date_of_Payment"].str.slice(start=6, stop=10).astype(int)
dfg['cmonth'] = (dfg["year"]-2013) * 12 + dfg["month"] - 7

dfg['category'] = dfg['Nature_of_Payment_or_Transfer_of_Value']

dfg['category'] = dfg['category'].replace(['Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program'] ,
                                            'non-cme_speaking')

dfg['category'] = dfg['category'].replace(['Compensation for serving as faculty or as a speaker for a medical education program',
                                             'Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program',
                                             'Compensation for serving as faculty or as a speaker for an accredited or certified continuing education program'],
                                            'cme_speaking')
dfg['category'] = dfg['category'].replace(['Consulting Fee'],
                                            'consulting')

dfg['category'] = dfg['category'].replace(['Education'],
                                            'education')

dfg['category'] = dfg['category'].replace(['Honoraria'],
                                            'honoraria')

dfg['category'] = dfg['category'].replace(['Food and Beverage'],
                                            'meal')

dfg['category'] = dfg['category'].replace(['Gift'],
                                            'gift')

dfg['category'] = dfg['category'].replace(['Travel and Lodging'],
                                            'travel')

dfg['category'] = dfg['category'].replace(['Charitable Contribution'],
                                            'charity')

dfg['category'] = dfg['category'].replace(['Entertainment'],
                                            'entertain')

dfg['category'] = dfg['category'].replace(['Grant'],
                                            'grant')

dfg['category'] = dfg['category'].replace(['Current or prospective ownership or investment interest'],
                                            'c_ownership')

dfg['category'] = dfg['category'].replace(['Royalty or License'],
                                            'royalty')

dfg['category'] = dfg['category'].replace(['Long term medical supply or device loan'],
                                            'device_loan')

In [ ]:
df_general_pivotc = dfg.pivot_table(index ='Covered_Recipient_Profile_ID',
                                   columns = ['category',  'Program_Year'],
                                  values= 'CPI_payment',
                                   aggfunc = ['sum'] ).fillna(0)
df_general_pivotc['index', 'index', 'covered_recipient_profile_id'] = df_general_pivotc.index


df_general_pivotc.columns = df_general_pivotc.columns.droplevel(0)
df_general_pivotc.columns = df_general_pivotc.columns.map(lambda x: ''.join([str(i) for i in x]))
df_general_pivotc  = df_general_pivotc.reset_index()


df_general_pivotc.drop('indexcovered_recipient_profile_id', axis=1, inplace=True)

df_general_pivotc.index = df_general_pivotc['Covered_Recipient_Profile_ID']
df_general_pivotc.drop('Covered_Recipient_Profile_ID', axis=1, inplace=True)

df_general_pivotc

,Acquisitions2021,Debt forgiveness2021,Debt forgiveness2022,c_ownership2014,c_ownership2015,c_ownership2016,c_ownership2017,c_ownership2018,c_ownership2019,c_ownership2020,...,royalty2021,travel2014,travel2015,travel2016,travel2017,travel2018,travel2019,travel2020,travel2021,travel2022
Covered_Recipient_Profile_ID,,,,,,,,,,,,,,,,,,,,,
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4939.888230,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,52.656997,0.0,0.0,0.0
207.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3072.311317,1609.696958,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11042754.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
11161690.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
11230513.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [ ]:
df_id2 = pd.concat([df_id2, df_general_pivotc],
                        axis = 1)
df_id2

,covered_recipient_profile_type,covered_recipient_npi,covered_recipient_profile_primary_specialty,has_multiple_ids,general2014,general2015,general2016,general2017,general2018,general2019,...,royalty2021,travel2014,travel2015,travel2016,travel2017,travel2018,travel2019,travel2020,travel2021,travel2022
4.0,Covered Recipient Physician,1003024811,Allopathic & Osteopathic Physicians|Internal M...,False,95.657796,364.557769,323.727958,89.866767,184.935089,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
39.0,Covered Recipient Physician,1003042169,Allopathic & Osteopathic Physicians|Internal M...,False,5382.920548,58.428022,27.350251,0.000000,0.000000,160.512265,...,0.0,4939.888230,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
64.0,Covered Recipient Physician,1003081944,Allopathic & Osteopathic Physicians|Internal M...,False,548.394349,1540.773623,942.138712,44.951292,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
196.0,Covered Recipient Physician,1003832437,Allopathic & Osteopathic Physicians|Internal M...,False,4196.729269,4962.183784,6031.760667,5222.468591,4987.152051,5914.628557,...,0.0,0.000000,0.000000,0.0,0.0,0.0,52.656997,0.0,0.0,0.0
207.0,Covered Recipient Physician,1003839101,Allopathic & Osteopathic Physicians|Internal M...,False,32217.342821,39515.358080,6775.180235,1118.684244,18.600731,164.358515,...,0.0,3072.311317,1609.696958,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11042754.0,Covered Recipient Physician/Covered Recipient ...,1891786422,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
11161690.0,Covered Recipient Physician/Covered Recipient ...,1992133086,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
11230513.0,Covered Recipient Physician/Covered Recipient ...,1245345180,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
11310124.0,Covered Recipient Physician,1750623849,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


### General payments excluding newly-added categories, ownership, and royalties

In [ ]:
# Payment amounts
df_general_pivotlc = dfg.query('(Nature_of_Payment_or_Transfer_of_Value != "Acquisitions") \
& (Nature_of_Payment_or_Transfer_of_Value != "Current or prospective ownership or investment interest") & \
(Nature_of_Payment_or_Transfer_of_Value != "Debt forgiveness") & \
 (Nature_of_Payment_or_Transfer_of_Value != "Long term medical supply or device loan" ) & \
 (Nature_of_Payment_or_Transfer_of_Value != "Royalty or License") ').pivot_table(index ='Covered_Recipient_Profile_ID',
                                   columns = 'Program_Year',
                                  values= 'Number_of_Payments_Included_in_Total_Amount',
                                   aggfunc = ['sum'] ).fillna(0)
df_general_pivotlc['id', 'covered_recipient_profile_id'] = df_general_pivotlc.index

df_general_pivotlc = df_general_pivotlc.reset_index(level=0, drop=True)
df_general_pivotlc.columns = df_general_pivotlc.columns.droplevel(0)
df_general_pivotlc = df_general_pivotlc.reset_index()
df_general_pivotlc.drop('index', axis=1, inplace=True)

df_general_pivotlc.rename(columns = {2014:'generallc2014',
                                   2015:'generallc2015',
                                   2016:'generallc2016',
                                   2017:'generallc2017',
                                   2018:'generallc2018',
                                   2019:'generallc2019',
                                   2020:'generallc2020',
                                   2021:'generallc2021',
                                    2022: 'generallc2022'}
                        , inplace = True)

df_general_pivotlc.index = df_general_pivotlc['covered_recipient_profile_id']
df_general_pivotlc.drop('covered_recipient_profile_id', axis=1, inplace=True)

df_id2 = pd.concat([df_id2, df_general_pivotlc],
                        axis = 1)
df_id2

,covered_recipient_profile_type,covered_recipient_npi,covered_recipient_profile_primary_specialty,has_multiple_ids,general2014,general2015,general2016,general2017,general2018,general2019,...,travel2022,generallc2014,generallc2015,generallc2016,generallc2017,generallc2018,generallc2019,generallc2020,generallc2021,generallc2022
4.0,Covered Recipient Physician,1003024811,Allopathic & Osteopathic Physicians|Internal M...,False,95.657796,364.557769,323.727958,89.866767,184.935089,0.000000,...,0.0,6.0,8.0,6.0,1.0,6.0,0.0,1.0,2.0,1.0
39.0,Covered Recipient Physician,1003042169,Allopathic & Osteopathic Physicians|Internal M...,False,5382.920548,58.428022,27.350251,0.000000,0.000000,160.512265,...,0.0,5.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0
64.0,Covered Recipient Physician,1003081944,Allopathic & Osteopathic Physicians|Internal M...,False,548.394349,1540.773623,942.138712,44.951292,0.000000,0.000000,...,0.0,29.0,61.0,26.0,1.0,0.0,0.0,0.0,0.0,0.0
196.0,Covered Recipient Physician,1003832437,Allopathic & Osteopathic Physicians|Internal M...,False,4196.729269,4962.183784,6031.760667,5222.468591,4987.152051,5914.628557,...,0.0,174.0,166.0,239.0,244.0,232.0,187.0,53.0,101.0,119.0
207.0,Covered Recipient Physician,1003839101,Allopathic & Osteopathic Physicians|Internal M...,False,32217.342821,39515.358080,6775.180235,1118.684244,18.600731,164.358515,...,0.0,60.0,46.0,31.0,20.0,1.0,3.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11042754.0,Covered Recipient Physician/Covered Recipient ...,1891786422,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,37.0
11161690.0,Covered Recipient Physician/Covered Recipient ...,1992133086,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,135.0,23.0
11230513.0,Covered Recipient Physician/Covered Recipient ...,1245345180,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0
11310124.0,Covered Recipient Physician,1750623849,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0


In [ ]:
#Payment number
df_general_pivotl = dfg.query('(Nature_of_Payment_or_Transfer_of_Value != "Acquisitions") \
& (Nature_of_Payment_or_Transfer_of_Value != "Current or prospective ownership or investment interest") & \
(Nature_of_Payment_or_Transfer_of_Value != "Debt forgiveness") & \
 (Nature_of_Payment_or_Transfer_of_Value != "Long term medical supply or device loan" ) & \
 (Nature_of_Payment_or_Transfer_of_Value != "Royalty or License") ').pivot_table(index ='Covered_Recipient_Profile_ID',
                                   columns = 'Program_Year',
                                  values= 'CPI_payment',
                                   aggfunc = ['sum'] ).fillna(0)
df_general_pivotl['id', 'covered_recipient_profile_id'] = df_general_pivotl.index

df_general_pivotl = df_general_pivotl.reset_index(level=0, drop=True)
df_general_pivotl.columns = df_general_pivotl.columns.droplevel(0)
df_general_pivotl = df_general_pivotl.reset_index()
df_general_pivotl.drop('index', axis=1, inplace=True)

df_general_pivotl.rename(columns = {2014:'generall2014',
                                    2015:'generall2015',
                                    2016:'generall2016',
                                    2017:'generall2017',
                                    2018:'generall2018',
                                    2019:'generall2019',
                                    2020:'generall2020',
                                    2021:'generall2021',
                                    2022: 'generall2022'}
                        , inplace = True)

df_general_pivotl.index = df_general_pivotl['covered_recipient_profile_id']
df_general_pivotl.drop('covered_recipient_profile_id', axis=1, inplace=True)

df_id2 = pd.concat([df_id2, df_general_pivotl],
                        axis = 1)
df_id2

,covered_recipient_profile_type,covered_recipient_npi,covered_recipient_profile_primary_specialty,has_multiple_ids,general2014,general2015,general2016,general2017,general2018,general2019,...,generallc2022,generall2014,generall2015,generall2016,generall2017,generall2018,generall2019,generall2020,generall2021,generall2022
4.0,Covered Recipient Physician,1003024811,Allopathic & Osteopathic Physicians|Internal M...,False,95.657796,364.557769,323.727958,89.866767,184.935089,0.000000,...,1.0,95.657796,364.557769,323.727958,89.866767,184.935089,0.000000,13.897129,29.020334,11.76
39.0,Covered Recipient Physician,1003042169,Allopathic & Osteopathic Physicians|Internal M...,False,5382.920548,58.428022,27.350251,0.000000,0.000000,160.512265,...,0.0,5382.920548,58.428022,27.350251,0.000000,0.000000,160.512265,0.000000,0.000000,0.00
64.0,Covered Recipient Physician,1003081944,Allopathic & Osteopathic Physicians|Internal M...,False,548.394349,1540.773623,942.138712,44.951292,0.000000,0.000000,...,0.0,548.394349,1540.773623,942.138712,44.951292,0.000000,0.000000,0.000000,0.000000,0.00
196.0,Covered Recipient Physician,1003832437,Allopathic & Osteopathic Physicians|Internal M...,False,4196.729269,4962.183784,6031.760667,5222.468591,4987.152051,5914.628557,...,119.0,4196.729269,4962.183784,6031.760667,5222.468591,4987.152051,5914.628557,1159.703573,2652.482270,2696.82
207.0,Covered Recipient Physician,1003839101,Allopathic & Osteopathic Physicians|Internal M...,False,32217.342821,39515.358080,6775.180235,1118.684244,18.600731,164.358515,...,1.0,32217.342821,39515.358080,5454.003384,1118.684244,18.600731,164.358515,0.000000,257.597314,50.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11042754.0,Covered Recipient Physician/Covered Recipient ...,1891786422,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,37.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,596.250677,1508.83
11161690.0,Covered Recipient Physician/Covered Recipient ...,1992133086,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28.269181,2329.629706,499.07
11230513.0,Covered Recipient Physician/Covered Recipient ...,1245345180,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,152.208249,86.50
11310124.0,Covered Recipient Physician,1750623849,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,11.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,204.97


### Associate research payments pivot by physician profile ID

In [ ]:
df_research_pivot1 = dfr.pivot_table(index ='Principal_Investigator_1_Profile_ID',
                                   columns = 'Program_Year',
                                   values= 'per_payment',
                                   aggfunc = ['sum']).fillna(0)

df_research_pivot2 = dfr.pivot_table(index ='Principal_Investigator_2_Profile_ID',
                                   columns = 'Program_Year',
                                  values= 'per_payment',
                                   aggfunc = ['sum'] ).fillna(0)

df_research_pivot3 = dfr.pivot_table(index ='Principal_Investigator_3_Profile_ID',
                                   columns = 'Program_Year',
                                  values= 'per_payment',
                                   aggfunc = ['sum']).fillna(0)

df_research_pivot4 = dfr.pivot_table(index ='Principal_Investigator_4_Profile_ID',
                                   columns = 'Program_Year',
                                  values= 'per_payment',
                                   aggfunc = ['sum']).fillna(0)

df_research_pivot5 = dfr.pivot_table(index ='Principal_Investigator_5_Profile_ID',
                                   columns = 'Program_Year',
                                  values= 'per_payment',
                                   aggfunc =['sum']).fillna(0)


In [ ]:
df_research_pivot = pd.concat([df_research_pivot1,
                               df_research_pivot2,
                               df_research_pivot3,
                               df_research_pivot4,
                               df_research_pivot5], axis = 0).fillna(0)
df_research_pivot['id', 'covered_recipient_profile_id'] = df_research_pivot.index
df_research_pivot = df_research_pivot.reset_index(level=0, drop=True)
df_research_pivot.columns = df_research_pivot.columns.droplevel(0)
df_research_pivot = df_research_pivot.reset_index()
df_research_pivot.drop('index', axis=1, inplace=True)
df_research_pivot.columns
df_research_pivot.rename(columns = {2014:'research2014',
                                   2015:'research2015',
                                   2016:'research2016',
                                   2017:'research2017',
                                   2018:'research2018',
                                   2019:'research2019',
                                   2020:'research2020',
                                   2021:'research2021',
                                    2022: 'research2022'}, inplace = True)
df_research_pivot.sort_values('covered_recipient_profile_id')

Program_Year,research2014,research2015,research2016,research2017,research2018,research2019,research2020,research2021,research2022,covered_recipient_profile_id
0,0.000000,380255.052443,1.967973e+06,1.558073e+06,0.000000,0.000000,0.000000,0.000000,0.00,567.0
1,414736.685473,317440.853646,7.025960e+05,1.595937e+05,197107.006580,429200.134849,803957.448871,77808.828654,73976.98,940.0
1264,30317.439062,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.00,1794.0
2,286727.715142,322017.884153,3.818169e+05,7.653254e+05,478845.545143,330575.547518,120638.062702,65459.904403,165379.72,1794.0
3,271464.783254,165051.199367,5.696310e+04,2.381988e+05,204339.451026,27348.258722,6845.178638,0.000000,0.00,2671.0
...,...,...,...,...,...,...,...,...,...,...
1259,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,4340.521753,32868.61,4216790.0
1260,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,27473.216067,18452.990580,0.000000,0.00,4216796.0
1261,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,11349.326576,9009.90,5691235.0
1262,0.000000,0.000000,0.000000e+00,0.000000e+00,1701.605609,0.000000,0.000000,0.000000,0.00,5701360.0


In [ ]:
df_research_pivot2 = df_research_pivot.pivot_table(index ='covered_recipient_profile_id',
                                                   values= ['research2014',
                                                            'research2015',
                                                            'research2016',
                                                            'research2017',
                                                            'research2018',
                                                            'research2019',
                                                            'research2020',
                                                            'research2021',
                                                            'research2022'],
                                                    aggfunc = ['sum'] ).fillna(0)

df_research_pivot2['id', 'covered_recipient_profile_id'] = df_research_pivot2.index

df_research_pivot2 = df_research_pivot2.reset_index(level=0, drop=True)
df_research_pivot2.columns = df_research_pivot2.columns.droplevel(0)
df_research_pivot2 = df_research_pivot2.reset_index()
df_research_pivot2.drop('index', axis=1, inplace=True)
df_research_pivot2.index = df_research_pivot2['covered_recipient_profile_id']

df_research_pivot2

Program_Year,research2014,research2015,research2016,research2017,research2018,research2019,research2020,research2021,research2022,covered_recipient_profile_id
covered_recipient_profile_id,,,,,,,,,,
567.0,0.000000,380255.052443,1.967973e+06,1.558073e+06,0.000000,0.000000,0.000000,0.000000,0.00,567.0
940.0,414736.685473,317440.853646,7.025960e+05,1.595937e+05,197107.006580,429200.134849,803957.448871,77808.828654,73976.98,940.0
1794.0,317045.154204,322017.884153,3.818169e+05,7.653254e+05,478845.545143,330575.547518,120638.062702,65459.904403,165379.72,1794.0
2671.0,271464.783254,165051.199367,5.696310e+04,2.381988e+05,204339.451026,27348.258722,6845.178638,0.000000,0.00,2671.0
2777.0,2199.263953,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.00,2777.0
...,...,...,...,...,...,...,...,...,...,...
4216790.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,4340.521753,32868.61,4216790.0
4216796.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,27473.216067,18452.990580,0.000000,0.00,4216796.0
5691235.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,11349.326576,9009.90,5691235.0


In [ ]:
df_research_pivot2 = df_research_pivot2[df_research_pivot2["covered_recipient_profile_id"].isin(list_id)]
df_research_pivot2.drop('covered_recipient_profile_id', axis=1, inplace=True)

<ipython-input-148-a6229108216a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_research_pivot2.drop('covered_recipient_profile_id', axis=1, inplace=True)


In [ ]:
df_id3 = pd.concat([df_id2, df_research_pivot2],
                        axis = 1).fillna(0)

### Direct research payments

In [ ]:
df_dresearch_pivot = dfrd.pivot_table(index ='Covered_Recipient_Profile_ID',
                                      columns = 'Program_Year',
                                      values= 'CPI_payment',
                                      aggfunc = ['sum'] ).fillna(0)
df_dresearch_pivot['id', 'covered_recipient_profile_id'] = df_dresearch_pivot.index

df_dresearch_pivot = df_dresearch_pivot.reset_index(level=0, drop=True)
df_dresearch_pivot.columns = df_dresearch_pivot.columns.droplevel(0)
df_dresearch_pivot = df_dresearch_pivot.reset_index()
df_dresearch_pivot.drop('index', axis=1, inplace=True)

df_dresearch_pivot.rename(columns = {2014:'researchd2014',
                                     2015:'researchd2015',
                                     2016:'researchd2016',
                                     2017:'researchd2017',
                                     2018:'researchd2018',
                                     2019:'researchd2019',
                                     2020:'researchd2020',
                                     2021:'researchd2021',
                                     2022: 'researchd2022'}
                        , inplace = True)

df_dresearch_pivot.index = df_dresearch_pivot['covered_recipient_profile_id']
df_dresearch_pivot.drop('covered_recipient_profile_id', axis=1, inplace=True)

df_dresearch_pivot

Program_Year,researchd2014,researchd2015,researchd2016,researchd2017,researchd2018,researchd2019,researchd2020,researchd2021,researchd2022
covered_recipient_profile_id,,,,,,,,,
940.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16961.508591,0.000000,0.00
1794.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,786.96
2671.0,0.000000,0.000000,2369.509648,26592.623335,55320.625685,5184.424815,0.000000,0.000000,0.00
2777.0,0.000000,3801.772637,3217.160133,10197.126021,7262.501642,582.180343,0.000000,0.000000,0.00
3954.0,3214.141491,44574.209867,26285.749311,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
...,...,...,...,...,...,...,...,...,...
2785202.0,0.000000,0.000000,0.000000,0.000000,0.000000,949.668949,0.000000,0.000000,0.00
2785537.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,156.24
3339381.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8042.468914,79.814018,0.00


In [ ]:
df_id3 = pd.concat([df_id3, df_dresearch_pivot],
                   axis = 1).fillna(0)

In [ ]:
df_id3

,covered_recipient_profile_type,covered_recipient_npi,covered_recipient_profile_primary_specialty,has_multiple_ids,general2014,general2015,general2016,general2017,general2018,general2019,...,research2022,researchd2014,researchd2015,researchd2016,researchd2017,researchd2018,researchd2019,researchd2020,researchd2021,researchd2022
4.0,Covered Recipient Physician,1003024811,Allopathic & Osteopathic Physicians|Internal M...,False,95.657796,364.557769,323.727958,89.866767,184.935089,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39.0,Covered Recipient Physician,1003042169,Allopathic & Osteopathic Physicians|Internal M...,False,5382.920548,58.428022,27.350251,0.000000,0.000000,160.512265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64.0,Covered Recipient Physician,1003081944,Allopathic & Osteopathic Physicians|Internal M...,False,548.394349,1540.773623,942.138712,44.951292,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196.0,Covered Recipient Physician,1003832437,Allopathic & Osteopathic Physicians|Internal M...,False,4196.729269,4962.183784,6031.760667,5222.468591,4987.152051,5914.628557,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
207.0,Covered Recipient Physician,1003839101,Allopathic & Osteopathic Physicians|Internal M...,False,32217.342821,39515.358080,6775.180235,1118.684244,18.600731,164.358515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11042754.0,Covered Recipient Physician/Covered Recipient ...,1891786422,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11161690.0,Covered Recipient Physician/Covered Recipient ...,1992133086,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11230513.0,Covered Recipient Physician/Covered Recipient ...,1245345180,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11310124.0,Covered Recipient Physician,1750623849,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Payments by month

In [ ]:
dfg['category'] = dfg['Nature_of_Payment_or_Transfer_of_Value']

dfg['category'] = dfg['category'].replace(['Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program'] ,
                                            'non-cme_speakingm')

dfg['category'] = dfg['category'].replace(['Compensation for serving as faculty or as a speaker for a medical education program',
                                             'Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program',
                                             'Compensation for serving as faculty or as a speaker for an accredited or certified continuing education program'],
                                            'cme_speakingm')


dfg['category'] = dfg['category'].replace(['Consulting Fee'],
                                            'consultingm')

dfg['category'] = dfg['category'].replace(['Education'],
                                            'educationm')

dfg['category'] = dfg['category'].replace(['Honoraria'],
                                            'honorariam')

dfg['category'] = dfg['category'].replace(['Food and Beverage'],
                                            'mealm')

dfg['category'] = dfg['category'].replace(['Gift'],
                                            'giftm')

dfg['category'] = dfg['category'].replace(['Travel and Lodging'],
                                            'travelm')

dfg['category'] = dfg['category'].replace(['Charitable Contribution',
                                             'Entertainment',
                                             'Grant'],
                                            'otherm')

dfg['category'] = dfg['category'].replace(['Current or prospective ownership or investment interest'],
                                            'c_ownershipm')

dfg['category'] = dfg['category'].replace(['Royalty or License'],
                                            'royaltym')

dfg['category'] = dfg['category'].replace(['Long term medical supply or device loan'],
                                            'device_loanm')

dfgl2 = dfg.query('category != "Acquisitions" & category != "Debt forgiveness" & category != "device_loan"')

dfg_wide_monthl = dfgl2.pivot_table(index ='Covered_Recipient_Profile_ID',
                                   columns = ['category','cmonth'],
                                  values= 'Total_Amount_of_Payment_USDollars',
                                   aggfunc = ['sum'] ).fillna(0)

dfg_wide_monthl.columns = dfg_wide_monthl.columns.droplevel(0)
dfg_wide_monthl.columns = dfg_wide_monthl.columns.map(lambda x: ''.join([str(i) for i in x]))
dfg_wide_monthl  = dfg_wide_monthl.reset_index()

dfg_wide_monthl.index = dfg_wide_monthl['Covered_Recipient_Profile_ID']
dfg_wide_monthl.drop('Covered_Recipient_Profile_ID', axis=1, inplace=True)
del dfgl2

dfg_wide_monthl

,c_ownershipm10,c_ownershipm23,c_ownershipm34,c_ownershipm47,c_ownershipm59,c_ownershipm70,c_ownershipm77,c_ownershipm82,c_ownershipm94,c_ownershipm102,...,travelm104,travelm105,travelm106,travelm107,travelm108,travelm109,travelm110,travelm111,travelm112,travelm113
Covered_Recipient_Profile_ID,,,,,,,,,,,,,,,,,,,,,
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
207.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11042754.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11161690.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11230513.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_idm = pd.concat([df_id[['covered_recipient_profile_type',
                           'covered_recipient_npi',
                           'covered_recipient_profile_primary_specialty',
                           'has_multiple_ids']], dfg_wide_monthl],
                        axis = 1)
df_idm

,covered_recipient_profile_type,covered_recipient_npi,covered_recipient_profile_primary_specialty,has_multiple_ids,c_ownershipm10,c_ownershipm23,c_ownershipm34,c_ownershipm47,c_ownershipm59,c_ownershipm70,...,travelm104,travelm105,travelm106,travelm107,travelm108,travelm109,travelm110,travelm111,travelm112,travelm113
4.0,Covered Recipient Physician,1003024811,Allopathic & Osteopathic Physicians|Internal M...,False,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39.0,Covered Recipient Physician,1003042169,Allopathic & Osteopathic Physicians|Internal M...,False,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64.0,Covered Recipient Physician,1003081944,Allopathic & Osteopathic Physicians|Internal M...,False,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196.0,Covered Recipient Physician,1003832437,Allopathic & Osteopathic Physicians|Internal M...,False,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
207.0,Covered Recipient Physician,1003839101,Allopathic & Osteopathic Physicians|Internal M...,False,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11042754.0,Covered Recipient Physician/Covered Recipient ...,1891786422,Allopathic & Osteopathic Physicians|Internal M...,False,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11161690.0,Covered Recipient Physician/Covered Recipient ...,1992133086,Allopathic & Osteopathic Physicians|Internal M...,False,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11230513.0,Covered Recipient Physician/Covered Recipient ...,1245345180,Allopathic & Osteopathic Physicians|Internal M...,False,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11310124.0,Covered Recipient Physician,1750623849,Allopathic & Osteopathic Physicians|Internal M...,False,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Save profile dataset

In [ ]:
npi_specialty.drop(npi_specialty.loc[:,'Healthcare Provider Taxonomy Code_1' : 'Healthcare Provider Primary Taxonomy Switch_15'].columns
                   , axis=1, inplace=True)

In [ ]:
del df_id
del df_id2

In [ ]:
df_id3['OPD_ID']= df_id3.index
df_id3.index= df_id3['covered_recipient_npi']
df_id3

,covered_recipient_profile_type,covered_recipient_npi,covered_recipient_profile_primary_specialty,has_multiple_ids,general2014,general2015,general2016,general2017,general2018,general2019,...,researchd2014,researchd2015,researchd2016,researchd2017,researchd2018,researchd2019,researchd2020,researchd2021,researchd2022,OPD_ID
covered_recipient_npi,,,,,,,,,,,,,,,,,,,,,
1003024811,Covered Recipient Physician,1003024811,Allopathic & Osteopathic Physicians|Internal M...,False,95.657796,364.557769,323.727958,89.866767,184.935089,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1003042169,Covered Recipient Physician,1003042169,Allopathic & Osteopathic Physicians|Internal M...,False,5382.920548,58.428022,27.350251,0.000000,0.000000,160.512265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0
1003081944,Covered Recipient Physician,1003081944,Allopathic & Osteopathic Physicians|Internal M...,False,548.394349,1540.773623,942.138712,44.951292,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0
1003832437,Covered Recipient Physician,1003832437,Allopathic & Osteopathic Physicians|Internal M...,False,4196.729269,4962.183784,6031.760667,5222.468591,4987.152051,5914.628557,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0
1003839101,Covered Recipient Physician,1003839101,Allopathic & Osteopathic Physicians|Internal M...,False,32217.342821,39515.358080,6775.180235,1118.684244,18.600731,164.358515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891786422,Covered Recipient Physician/Covered Recipient ...,1891786422,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11042754.0
1992133086,Covered Recipient Physician/Covered Recipient ...,1992133086,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11161690.0
1245345180,Covered Recipient Physician/Covered Recipient ...,1245345180,Allopathic & Osteopathic Physicians|Internal M...,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11230513.0


In [ ]:
npi_specialty.index= npi_specialty['NPI']
df_npi_merge = pd.concat([npi_specialty, df_id3],
                         axis = 1).fillna(0)
df_npi_merge

,NPI,Entity Type Code,Replacement NPI,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,...,researchd2014,researchd2015,researchd2016,researchd2017,researchd2018,researchd2019,researchd2020,researchd2021,researchd2022,OPD_ID
1316940141,1316940141,1.0,0.0,0.0,PASSARO,MAUREEN,D,0.0,0.0,DONNELLY,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1169529.0
1609879709,1609879709,1.0,0.0,0.0,FRUITERMAN,MARK,LESTER,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,299271.0
1396748463,1396748463,1.0,0.0,0.0,KOLATKAR,NIKHEEL,SHRINIVAS,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1285637553,1285637553,1.0,0.0,0.0,PENABAD,JESUS,L,0.0,0.0,0,...,0.0,0.0,0.0,0.0,759.961076,0.0,0.0,0.0,0.0,149348.0
1972506244,1972506244,1.0,0.0,0.0,LOMAN,LUSIANA,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,362364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396784344,1396784344,1.0,0.0,0.0,STEINSAPIR,JAIME,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,108763.0
1689678286,1689678286,1.0,0.0,0.0,CAPLAN,ROBERT,H,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1033179940,1033179940,1.0,0.0,0.0,ELLYIN,FREDRICK,M,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,620336.0
1124184858,1124184858,1.0,0.0,0.0,TUCK,MICHAEL,LAWRENCE,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,6198.0


In [ ]:
del df_id3

In [ ]:
df_idm['OPD_ID']= df_idm.index
df_idm.index= df_idm['covered_recipient_npi']

df_npi_mergem = pd.concat([npi_specialty, df_idm],
                          axis = 1)
df_npi_mergem

,NPI,Entity Type Code,Replacement NPI,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,...,travelm105,travelm106,travelm107,travelm108,travelm109,travelm110,travelm111,travelm112,travelm113,OPD_ID
1316940141,1316940141,1.0,NaN,NaN,PASSARO,MAUREEN,D,NaN,NaN,DONNELLY,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1169529.0
1609879709,1609879709,1.0,NaN,NaN,FRUITERMAN,MARK,LESTER,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,299271.0
1396748463,1396748463,1.0,NaN,NaN,KOLATKAR,NIKHEEL,SHRINIVAS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1285637553,1285637553,1.0,NaN,NaN,PENABAD,JESUS,L,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149348.0
1972506244,1972506244,1.0,NaN,NaN,LOMAN,LUSIANA,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,362364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396784344,1396784344,1.0,NaN,NaN,STEINSAPIR,JAIME,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108763.0
1689678286,1689678286,1.0,NaN,NaN,CAPLAN,ROBERT,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1033179940,1033179940,1.0,NaN,NaN,ELLYIN,FREDRICK,M,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,620336.0
1124184858,1124184858,1.0,NaN,NaN,TUCK,MICHAEL,LAWRENCE,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6198.0


In [ ]:
df_npi_mergem

,NPI,Entity Type Code,Replacement NPI,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,...,travelm105,travelm106,travelm107,travelm108,travelm109,travelm110,travelm111,travelm112,travelm113,OPD_ID
1316940141,1316940141,1.0,NaN,NaN,PASSARO,MAUREEN,D,NaN,NaN,DONNELLY,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1169529.0
1609879709,1609879709,1.0,NaN,NaN,FRUITERMAN,MARK,LESTER,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,299271.0
1396748463,1396748463,1.0,NaN,NaN,KOLATKAR,NIKHEEL,SHRINIVAS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1285637553,1285637553,1.0,NaN,NaN,PENABAD,JESUS,L,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149348.0
1972506244,1972506244,1.0,NaN,NaN,LOMAN,LUSIANA,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,362364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396784344,1396784344,1.0,NaN,NaN,STEINSAPIR,JAIME,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108763.0
1689678286,1689678286,1.0,NaN,NaN,CAPLAN,ROBERT,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1033179940,1033179940,1.0,NaN,NaN,ELLYIN,FREDRICK,M,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,620336.0
1124184858,1124184858,1.0,NaN,NaN,TUCK,MICHAEL,LAWRENCE,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6198.0


In [ ]:
del dfg_wide_monthl

In [ ]:
df_npi_merge

,NPI,Entity Type Code,Replacement NPI,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,...,researchd2014,researchd2015,researchd2016,researchd2017,researchd2018,researchd2019,researchd2020,researchd2021,researchd2022,OPD_ID
1316940141,1316940141,1.0,0.0,0.0,PASSARO,MAUREEN,D,0.0,0.0,DONNELLY,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1169529.0
1609879709,1609879709,1.0,0.0,0.0,FRUITERMAN,MARK,LESTER,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,299271.0
1396748463,1396748463,1.0,0.0,0.0,KOLATKAR,NIKHEEL,SHRINIVAS,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1285637553,1285637553,1.0,0.0,0.0,PENABAD,JESUS,L,0.0,0.0,0,...,0.0,0.0,0.0,0.0,759.961076,0.0,0.0,0.0,0.0,149348.0
1972506244,1972506244,1.0,0.0,0.0,LOMAN,LUSIANA,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,362364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396784344,1396784344,1.0,0.0,0.0,STEINSAPIR,JAIME,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,108763.0
1689678286,1689678286,1.0,0.0,0.0,CAPLAN,ROBERT,H,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1033179940,1033179940,1.0,0.0,0.0,ELLYIN,FREDRICK,M,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,620336.0
1124184858,1124184858,1.0,0.0,0.0,TUCK,MICHAEL,LAWRENCE,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,6198.0


In [ ]:
df_npi_merge.to_csv(path_out_profile + "npi_dataset_payment2014-2022.csv", index=None)

In [ ]:
df_npi_mergem.to_csv(path_out_profile + "npi_dataset_payment2014-2022month.csv", index=None)

In [ ]:
del df_dresearch_pivot
del df_general_pivot
del df_general_pivotc
del df_general_pivotl
del df_research_pivot
del df_research_pivot1
del df_research_pivot2
del df_research_pivot3
del df_research_pivot4
del df_research_pivot5

## Pivot table by pharmaceutical companies

### Pivot table by company for associate research payments

In [ ]:
dfr.loc[dfr["Principal_Investigator_1_Profile_ID"].isin(list_id),  "PI1"]=1
dfr.loc[~dfr["Principal_Investigator_1_Profile_ID"].isin(list_id),  "PI1"]=0

dfr.loc[dfr["Principal_Investigator_2_Profile_ID"].isin(list_id),  "PI2"]=1
dfr.loc[~dfr["Principal_Investigator_2_Profile_ID"].isin(list_id),  "PI2"]=0

dfr.loc[dfr["Principal_Investigator_3_Profile_ID"].isin(list_id),  "PI3"]=1
dfr.loc[~dfr["Principal_Investigator_3_Profile_ID"].isin(list_id),  "PI3"]=0

dfr.loc[dfr["Principal_Investigator_4_Profile_ID"].isin(list_id),  "PI4"]=1
dfr.loc[~dfr["Principal_Investigator_4_Profile_ID"].isin(list_id),  "PI4"]=0

dfr.loc[dfr["Principal_Investigator_5_Profile_ID"].isin(list_id),  "PI5"]=1
dfr.loc[~dfr["Principal_Investigator_5_Profile_ID"].isin(list_id),  "PI5"]=0

dfr["PI"] = dfr["PI1"] +dfr["PI2"] +dfr["PI3"] +dfr["PI4"] +dfr["PI5"]

In [ ]:
dfr["payment_DMdoc"] = dfr['per_payment'] *dfr["PI"]

In [ ]:
df_pharmaryr = dfr.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                             columns = 'Program_Year',
                             values= 'PI_OPD_per_payment',
                             aggfunc = ['sum']).fillna(0)
df_pharmaryr["index"] = df_pharmaryr.index

In [ ]:
df_pharmar = dfr.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                             values= 'PI_OPD_per_payment',
                             aggfunc = ['sum']).fillna(0)
df_pharmar["index", 'ID'] = df_pharmar.index
df_pharmar = df_pharmar.reset_index(level=0, drop=True)
df_pharmar.columns = df_pharmar.columns.droplevel(0)
df_pharmar = df_pharmar.reset_index()
df_pharmar.drop('index', axis=1, inplace=True)
df_pharmar = df_pharmar.rename(columns={'PI_OPD_per_payment': 'Associate research'})
df_pharmar.index = df_pharmar['ID']
df_pharmar.drop('ID', axis=1, inplace=True)
df_pharmar

,Associate research
ID,
100000000053,5.609140e+07
100000000058,5.999739e+03
100000000062,2.668489e+03
100000000066,1.220069e+06
100000000067,1.469072e+08
...,...
100000961851,8.856612e+06
100000966832,3.411408e+05
100000966862,4.826794e+04


### Number of recipients by company

In [ ]:
def count_nonzero(values):
    return (values != 0).sum()
df_pharmar_n1 = dfr.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                                columns = "Principal_Investigator_1_Profile_ID",
                                values= 'PI_OPD_per_payment',
                                aggfunc = ['sum'], fill_value=0)
df_pharmar_n2 = dfr.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                                columns = "Principal_Investigator_2_Profile_ID",
                                values= 'PI_OPD_per_payment',
                                aggfunc = ['sum'], fill_value=0)
df_pharmar_n3 = dfr.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                                columns = "Principal_Investigator_3_Profile_ID",
                                values= 'PI_OPD_per_payment',
                                aggfunc = ['sum'], fill_value=0)
df_pharmar_n4 = dfr.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                                columns = "Principal_Investigator_4_Profile_ID",
                                values= 'PI_OPD_per_payment',
                                aggfunc = ['sum'], fill_value=0)
df_pharmar_n5 = dfr.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                                columns = "Principal_Investigator_5_Profile_ID",
                                values= 'PI_OPD_per_payment',
                                aggfunc = ['sum'], fill_value=0)

### Pivot table by company for direct research payments

In [ ]:
df_pharmard = dfrd.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                              values= 'CPI_payment',
                              aggfunc = ['sum']).fillna(0)

df_pharmard["index", "ID"] = df_pharmard.index
df_pharmard = df_pharmard.reset_index(level=0, drop=True)
df_pharmard.columns = df_pharmard.columns.droplevel(0)
df_pharmard = df_pharmard.reset_index()
df_pharmard.drop('index', axis=1, inplace=True)
df_pharmard = df_pharmard.rename(columns={'CPI_payment': 'Direct research'})

df_pharmard.index = df_pharmard['ID']

df_pharmard.drop('ID', axis=1, inplace=True)
df_pharmard

,Direct research
ID,
100000000053,6.233069e+05
100000000056,3.371016e+04
100000000058,4.087836e+03
100000000066,1.206539e+03
100000000067,3.417424e+06
...,...
100000316825,5.919581e+03
100000576811,1.705427e+05
100000801820,4.634220e+04


In [ ]:
dfrd

,Change_Type,Covered_Recipient_Type,Noncovered_Recipient_Entity_Name,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,...,Associated_Device_or_Medical_Supply_PDI_3,Covered_or_Noncovered_Indicator_4,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4,Product_Category_or_Therapeutic_Area_4,Associated_Device_or_Medical_Supply_PDI_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Associated_Device_or_Medical_Supply_PDI_5,CPI_payment
0,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,222358.0,OPADA,NaN,ALZOHAILI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741.724959
1,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,168576.0,HOWARD,NaN,HARRISON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.362083
2,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,168576.0,HOWARD,NaN,HARRISON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.362083
3,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,156997.0,ASHWINI,NaN,GORE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.308244
4,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,NaN,198071.0,ALEX,NaN,MANZANO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1607.070745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11600,NEW,Covered Recipient Physician,NaN,NaN,NaN,NaN,553080.0,PAOLA,NaN,MANSILLA-LETELIER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.300000
11601,NEW,Covered Recipient Physician,NaN,NaN,NaN,NaN,932767.0,MARTHA,YANETTY,GOMEZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15961.000000
11602,NEW,Covered Recipient Physician,NaN,NaN,NaN,NaN,263633.0,GEORGE,L,BAKRIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7063.610000
11603,NEW,Covered Recipient Physician,NaN,NaN,NaN,NaN,333964.0,ALTAGRACIA,A,ALCANTARA GONZALEZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,520.000000


In [ ]:
def count_nonzero(values):
    return (values != 0).sum()
dfrd_pharmar_n1 = dfrd.pivot_table(index = 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                                   columns = "Covered_Recipient_Profile_ID",
                                   values= 'CPI_payment',
                                aggfunc = ['sum'], fill_value=0)
dfrd_pharmar_n1

sum            \
Covered_Recipient_Profile_ID                       940.0     1794.0      
Applicable_Manufacturer_or_Applicable_GPO_Makin...                       
100000000053                                             0.0       0.0   
100000000056                                             0.0       0.0   
100000000058                                             0.0       0.0   
100000000066                                             0.0       0.0   
100000000067                                             0.0       0.0   
...                                                      ...       ...   
100000316825                                             0.0       0.0   
100000576811                                             0.0       0.0   
100000801820                                             0.0       0.0   
100000806824                                             0.0       0.0   
100001176927                                             0.0       0.0   

                                                                        \
Covered_Recipient_Profile_ID                       2671.0    2777.0      
Applicable_Manufacturer_or_Applicable_GPO_Makin...                       
100000000053                                             0.0       0.0   
100000000056                                             0.0       0.0   
100000000058                                             0.0       0.0   
100000000066                                             0.0       0.0   
100000000067                                             0.0       0.0   
...                                                      ...       ...   
100000316825                                             0.0       0.0   
100000576811                                             0.0       0.0   
100000801820                                             0.0       0.0   
100000806824                                             0.0       0.0   
100001176927                                             0.0       0.0   

                                                                         \
Covered_Recipient_Profile_ID                       3954.0     4305.0      
Applicable_Manufacturer_or_Applicable_GPO_Makin...                        
100000000053                                             0.0   0.000000   
100000000056                                             0.0   0.000000   
100000000058                                             0.0   0.000000   
100000000066                                             0.0   0.000000   
100000000067                                             0.0  23.134368   
...                                                      ...        ...   
100000316825                                             0.0   0.000000   
100000576811                                             0.0   0.000000   
100000801820                                             0.0   0.000000   
100000806824                                             0.0   0.000000   
100001176927                                             0.0   0.000000   

                                                                              \
Covered_Recipient_Profile_ID                          6297.0       7523.0      
Applicable_Manufacturer_or_Applicable_GPO_Makin...                             
100000000053                                           0.000000     0.000000   
100000000056                                           0.000000     0.000000   
100000000058                                           0.000000     0.000000   
100000000066                                           0.000000     0.000000   
100000000067                                        7530.195999  2009.112057   
...                                                         ...          ...   
100000316825                                           0.000000     0.000000   
100000576811                                           0.000000     0.000000   
100000801820                                        

In [ ]:
df_assor_n_pivot = pd.concat([df_pharmar_n1,
                              df_pharmar_n2,
                              df_pharmar_n3,
                              df_pharmar_n4,
                              df_pharmar_n5,
                              dfrd_pharmar_n1]).fillna(0)

df_assor_n_pivot['id', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID'] = df_assor_n_pivot.index
count_nonzero = lambda row: (row != 0).sum()
df_assor_n_pivot['NonZeroCount'] = df_assor_n_pivot.apply(count_nonzero, axis=1)
df_assor_n_pivot.sort_values('Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID')

df_assor_n_pivot.to_csv(path_out2 + "number_physicians_by company.csv", index=None)

df_assor_n_pivot

sum                      \
                                                   567.0 940.0        1794.0   
Applicable_Manufacturer_or_Applicable_GPO_Makin...                             
100000000053                                         0.0   0.0      0.000000   
100000000058                                         0.0   0.0      0.000000   
100000000062                                         0.0   0.0      0.000000   
100000000066                                         0.0   0.0      0.000000   
100000000067                                         0.0   0.0  66866.789909   
...                                                  ...   ...           ...   
100000316825                                         0.0   0.0      0.000000   
100000576811                                         0.0   0.0      0.000000   
100000801820                                         0.0   0.0      0.000000   
100000806824                                         0.0   0.0      0.000000   
100001176927                                         0.0   0.0      0.000000   

                                                                         \
                                                          2671.0 2777.0   
Applicable_Manufacturer_or_Applicable_GPO_Makin...                        
100000000053                                            0.000000    0.0   
100000000058                                            0.000000    0.0   
100000000062                                            0.000000    0.0   
100000000066                                            0.000000    0.0   
100000000067                                        72901.928527    0.0   
...                                                          ...    ...   
100000316825                                            0.000000    0.0   
100000576811                                            0.000000    0.0   
100000801820                                            0.000000    0.0   
100000806824                                            0.000000    0.0   
100001176927                                            0.000000    0.0   

                                                                         \
                                                   3569.0 3697.0 3954.0   
Applicable_Manufacturer_or_Applicable_GPO_Makin...                        
100000000053                                          0.0    0.0    0.0   
100000000058                                          0.0    0.0    0.0   
100000000062                                          0.0    0.0    0.0   
100000000066                                          0.0    0.0    0.0   
100000000067                                          0.0    0.0    0.0   
...                                                   ...    ...    ...   
100000316825                                          0.0    0.0    0.0   
100000576811                                          0.0    0.0    0.0   
100000801820                                          0.0    0.0    0.0   
100000806824                                          0.0    0.0    0.0   
100001176927                                          0.0    0.0    0.0   

                                                                          ...  \
                                                           4305.0 4481.0  ...   
Applicable_Manufacturer_or_Applicable_GPO_Makin...                        ...   
100000000053                                             0.000000    0.0  ...   
100000000058                                             0.000000    0.0  ...   
100000000062                                             0.000000    0.0  ...   
100000000066                                             0.000000    0.0  ...   
100000000067                                        133954.495641    0.0  ...   
...                                                           ...    ...  ...   
100000316825                                             0.000000    0.0  ...   
100000576811        

### Pivot table by company for general payments

In [ ]:
df_pharmagyr = dfg.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                               columns = 'Program_Year',
                               values= 'CPI_payment',
                               aggfunc = ['sum']).fillna(0)
df_pharmagyr["index"] = df_pharmagyr.index

In [ ]:
df_pharmag = dfg.pivot_table(index ='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                                   columns = 'Nature_of_Payment_or_Transfer_of_Value',
                                  values= 'CPI_payment',
                                   aggfunc = ['sum']).fillna(0)
df_pharmag["index", "ID"] = df_pharmag.index
df_pharmag = df_pharmag.reset_index(level=0, drop=True)
df_pharmag.columns = df_pharmag.columns.droplevel(0)
df_pharmag= df_pharmag.reset_index()
df_pharmag.drop('index', axis=1, inplace=True)
df_pharmag.index = df_pharmag['ID']

df_pharmag.drop('ID', axis=1, inplace=True)
df_pharmag

Nature_of_Payment_or_Transfer_of_Value,Acquisitions,Charitable Contribution,"Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program",Compensation for serving as faculty or as a speaker for a medical education program,Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program,Compensation for serving as faculty or as a speaker for an accredited or certified continuing education program,Consulting Fee,Current or prospective ownership or investment interest,Debt forgiveness,Education,Entertainment,Food and Beverage,Gift,Grant,Honoraria,Long term medical supply or device loan,Royalty or License,Travel and Lodging
ID,,,,,,,,,,,,,,,,,,
100000000053,0.0,0.0,1.205153e+07,0.0,1852.846788,0.0,2.521617e+06,0.0,0.0,100242.501871,0.0,1.426476e+06,0.0,6181.041329,0.0,0.0,0.0,3.114919e+06
100000000055,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,1.199645e+04,0.0,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.000000e+00
100000000056,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,1.447925e+04,0.0,0.0,0.000000,0.0,9.234289e+02,0.0,0.000000,0.0,0.0,0.0,2.559401e+04
100000000057,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,0.0,2.427746e+01,0.0,0.000000,0.0,0.0,0.0,0.000000e+00
100000000058,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,3.975646e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100001176895,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,0.0,1.243900e+02,0.0,0.000000,0.0,0.0,0.0,0.000000e+00
100001176921,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,0.0,3.015900e+02,0.0,0.000000,0.0,0.0,0.0,0.000000e+00
100001176927,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,7.041660e+03,0.0,0.0,1312.500000,0.0,1.962700e+02,0.0,0.000000,0.0,12000.0,0.0,0.000000e+00


### save pivot for pharma

In [ ]:
df_pharma = pd.concat([df_pharmag,
                       df_pharmar,
                       df_pharmard],
                        axis = 1).fillna(0)
df_pharma['ID']= df_pharma.index
df_pharma

,Acquisitions,Charitable Contribution,"Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program",Compensation for serving as faculty or as a speaker for a medical education program,Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program,Compensation for serving as faculty or as a speaker for an accredited or certified continuing education program,Consulting Fee,Current or prospective ownership or investment interest,Debt forgiveness,Education,...,Food and Beverage,Gift,Grant,Honoraria,Long term medical supply or device loan,Royalty or License,Travel and Lodging,Associate research,Direct research,ID
ID,,,,,,,,,,,,,,,,,,,,,
100000000053,0.0,0.0,1.205153e+07,0.0,1852.846788,0.0,2.521617e+06,0.0,0.0,100242.501871,...,1.426476e+06,0.0,6181.041329,0.0,0.0,0.0,3.114919e+06,5.609140e+07,623306.863500,100000000053
100000000055,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,1.199645e+04,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,100000000055
100000000056,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,1.447925e+04,0.0,0.0,0.000000,...,9.234289e+02,0.0,0.000000,0.0,0.0,0.0,2.559401e+04,0.000000e+00,33710.163199,100000000056
100000000057,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,...,2.427746e+01,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,100000000057
100000000058,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,3.975646e+02,5.999739e+03,4087.836063,100000000058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000961851,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,8.856612e+06,0.000000,100000961851
100000966862,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,4.826794e+04,0.000000,100000966862
100000000143,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,8400.035166,100000000143


In [ ]:
df_pharma.to_csv(path_out_profile + "pivot_pay_pharma.csv", index=None)

### Company payment by submitting companies

In [ ]:
dfr.loc[dfr["Principal_Investigator_1_Profile_ID"].isin(list_id),  "PI1"]=1
dfr.loc[~dfr["Principal_Investigator_1_Profile_ID"].isin(list_id),  "PI1"]=0

dfr.loc[dfr["Principal_Investigator_2_Profile_ID"].isin(list_id),  "PI2"]=1
dfr.loc[~dfr["Principal_Investigator_2_Profile_ID"].isin(list_id),  "PI2"]=0

dfr.loc[dfr["Principal_Investigator_3_Profile_ID"].isin(list_id),  "PI3"]=1
dfr.loc[~dfr["Principal_Investigator_3_Profile_ID"].isin(list_id),  "PI3"]=0

dfr.loc[dfr["Principal_Investigator_4_Profile_ID"].isin(list_id),  "PI4"]=1
dfr.loc[~dfr["Principal_Investigator_4_Profile_ID"].isin(list_id),  "PI4"]=0

dfr.loc[dfr["Principal_Investigator_5_Profile_ID"].isin(list_id),  "PI5"]=1
dfr.loc[~dfr["Principal_Investigator_5_Profile_ID"].isin(list_id),  "PI5"]=0

dfr["PI"] = dfr["PI1"] +dfr["PI2"] +dfr["PI3"] +dfr["PI4"] +dfr["PI5"]

dfr["payment_DMdoc"] = dfr['per_payment'] *dfr["PI"]

df_pharmar2 = dfr.pivot_table(index ='Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
                             values= 'payment_DMdoc',
                             aggfunc = ['sum']).fillna(0)
df_pharmar2["index", 'ID'] = df_pharmar2.index
df_pharmar2 = df_pharmar2.reset_index(level=0, drop=True)
df_pharmar2.columns = df_pharmar2.columns.droplevel(0)
df_pharmar2 = df_pharmar2.reset_index()
df_pharmar2.drop('index', axis=1, inplace=True)
df_pharmar2 = df_pharmar2.rename(columns={'payment_DMdoc': 'Research'})
df_pharmar2.index = df_pharmar2['ID']
df_pharmar2.drop('ID', axis=1, inplace=True)
df_pharmar2

,Research
ID,
ABBVIE INC.,3.272817e+06
ADAPT Pharma Inc.,9.371226e+02
AKEBIA THERAPEUTICS INC,1.290693e+05
"AMAG Pharmaceuticals, Inc.",2.090016e+05
AbbVie Inc.,2.412545e+06
...,...
ViiV Healthcare Company,1.040773e+06
"Visby Medical, Inc.",4.826794e+04
Welch Allyn,1.181320e+05


In [ ]:
df_pharmag2 = dfg.pivot_table(index ='Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
                                   columns = 'Nature_of_Payment_or_Transfer_of_Value',
                                  values= 'CPI_payment',
                                   aggfunc = ['sum']).fillna(0)
df_pharmag2["index", "ID"] = df_pharmag2.index
df_pharmag2 = df_pharmag2.reset_index(level=0, drop=True)
df_pharmag2.columns = df_pharmag2.columns.droplevel(0)
df_pharmag2= df_pharmag2.reset_index()
df_pharmag2.drop('index', axis=1, inplace=True)
df_pharmag2.index = df_pharmag2['ID']

df_pharmag2.drop('ID', axis=1, inplace=True)
df_pharmag2

Nature_of_Payment_or_Transfer_of_Value,Acquisitions,Charitable Contribution,"Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program",Compensation for serving as faculty or as a speaker for a medical education program,Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program,Compensation for serving as faculty or as a speaker for an accredited or certified continuing education program,Consulting Fee,Current or prospective ownership or investment interest,Debt forgiveness,Education,Entertainment,Food and Beverage,Gift,Grant,Honoraria,Long term medical supply or device loan,Royalty or License,Travel and Lodging
ID,,,,,,,,,,,,,,,,,,
"A-dec, Inc.",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,75.754857,0.0,0.000000,0.0,0.0,0.000000
ABB Con-Cise Optical Group LLC,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,63.739202,0.000000,0.0,0.000000,0.0,0.0,0.000000
ABBVIE INC.,0.0,0.0,45150.000000,0.0,0.0,0.0,68225.000000,0.0,0.0,41.76,0.0,90928.784181,0.000000,0.0,0.000000,0.0,0.0,68.320000
ABIOMED,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,2643.376162,0.000000,0.0,0.000000,0.0,0.0,0.000000
ACADIA Pharmaceuticals Inc,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,1848.868533,0.000000,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"diaDexus, Inc.",0.0,0.0,2469.485311,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,3032.411438,0.000000,0.0,5810.178849,0.0,0.0,0.000000
"iRhythm Technologies, Inc.",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,343.461169,0.000000,0.0,0.000000,0.0,0.0,0.000000
iScreen Vision Inc.,0.0,0.0,0.000000,0.0,0.0,0.0,432.672893,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000


In [ ]:
df_pharma2 = pd.concat([df_pharmag2, df_pharmar2],
                        axis = 1).fillna(0)
df_pharma2['ID']= df_pharma2.index
df_pharma2

,Acquisitions,Charitable Contribution,"Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program",Compensation for serving as faculty or as a speaker for a medical education program,Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program,Compensation for serving as faculty or as a speaker for an accredited or certified continuing education program,Consulting Fee,Current or prospective ownership or investment interest,Debt forgiveness,Education,Entertainment,Food and Beverage,Gift,Grant,Honoraria,Long term medical supply or device loan,Royalty or License,Travel and Lodging,Research,ID
ID,,,,,,,,,,,,,,,,,,,,
"A-dec, Inc.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,75.754857,0.0,0.0,0.0,0.0,0.00,0.000000e+00,"A-dec, Inc."
ABB Con-Cise Optical Group LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,63.739202,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000e+00,ABB Con-Cise Optical Group LLC
ABBVIE INC.,0.0,0.0,45150.0,0.0,0.0,0.0,68225.0,0.0,0.0,41.76,0.0,90928.784181,0.000000,0.0,0.0,0.0,0.0,68.32,3.272817e+06,ABBVIE INC.
ABIOMED,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2643.376162,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000e+00,ABIOMED
ACADIA Pharmaceuticals Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1848.868533,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000e+00,ACADIA Pharmaceuticals Inc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Soleno Therapeutics, Inc.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,1.113240e+04,"Soleno Therapeutics, Inc."
"Takeda Development Center Americas, Inc.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,4.486663e+04,"Takeda Development Center Americas, Inc."
Valeant Pharmaceuticals North America LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,7.316125e+04,Valeant Pharmaceuticals North America LLC


In [ ]:
df_pharma2.to_csv(path_out_profile + "pivot_pay_pharma2.csv", index=None)

## Pivot table by payment nature

### Payment nature

In [ ]:
dfg['category2'] = dfg['Nature_of_Payment_or_Transfer_of_Value']
dfg['category2'] = dfg['category2'].replace(['Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program'] ,
                                            'non-cme_speaking')
dfg['category2'] = dfg['category2'].replace(['Compensation for serving as faculty or as a speaker for a medical education program',
                                             'Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program',
                                             'Compensation for serving as faculty or as a speaker for an accredited or certified continuing education program'],
                                            'cme_speaking')

In [ ]:
nature_g = dfg.pivot_table(index = 'Program_Year',
                           columns = 'category2',
                           values= ['CPI_payment',
                                    'Number_of_Payments_Included_in_Total_Amount'],
                           aggfunc = ['sum'] ).fillna(0)
nature_g.to_csv(path_out1 + "category_general.csv", index=None)
nature_g

sum                                         \
               CPI_payment                                          
category2     Acquisitions Charitable Contribution Consulting Fee   
Program_Year                                                        
2014              0.000000              618.104133   1.305542e+07   
2015              0.000000                0.000000   1.013485e+07   
2016              0.000000             1829.040403   9.788187e+06   
2017              0.000000                0.000000   1.005507e+07   
2018              0.000000                0.000000   9.180252e+06   
2019              0.000000                0.000000   8.953780e+06   
2020              0.000000              565.383620   4.287094e+06   
2021          20264.703608                0.000000   4.530726e+06   
2022              0.000000             2770.580000   4.407484e+06   

                                                                      \
                                                                       
category2    Current or prospective ownership or investment interest   
Program_Year                                                           
2014                                              287415.949370        
2015                                              400481.371885        
2016                                              399119.198484        
2017                                              363387.320873        
2018                                              387587.246283        
2019                                              382936.532523        
2020                                              376676.094341        
2021                                              332519.888013        
2022                                                5589.840000        

                                                                              \
                                                                               
category2    Debt forgiveness      Education Entertainment Food and Beverage   
Program_Year                                                                   
2014                 0.000000  254844.111466     76.088619      7.006419e+06   
2015                 0.000000  304491.440655    132.253286      7.621958e+06   
2016                 0.000000  224892.186825      0.000000      7.435228e+06   
2017                 0.000000  138826.699340     83.909079      7.442825e+06   
2018                 0.000000   88194.481057    247.007454      6.629931e+06   
2019                 0.000000   84903.249827    136.564778      5.856730e+06   
2020                 0.000000   54009.129656      0.000000      2.856218e+06   
2021                13.629945   59853.774247     79.922021      3.422812e+06   
2022              5554.280000   54518.630000     25.240000      3.763120e+06   

                                           ...  \
                                           ...   
category2             Gift          Grant  ...   
Program_Year                               ...   
2014           6702.053664  227459.848481  ...   
2015           1783.882104  114622.778231  ...   
2016          14392.670157  273940.575630  ...   
2017           7586.703595   37657.088419  ...   
2018          60790.942218  133216.665087  ...   
2019          45281.652348   10665.331421  ...   
2020           5989.165222    3615.628248  ...   
2021           5981.115637   10451.510674  ...   
2022          14897.830000  338477.030000  ...   

                                                                            \
             Number_of_Payments_Included_in_Total_Amount                     
category2                                  Entertainment Food and Beverage   
Program_Year                                                                 
2014                                                 3.0          239955.0   
2015                                                 3.0          273434.0   
2016                        

In [ ]:
df_pivot_g_nature = dfg.pivot_table(index ='Covered_Recipient_Profile_ID',
                                   columns = 'category2',
                                   values= 'CPI_payment',
                                   aggfunc = ['sum'] ).fillna(0)

df_pivot_g_nature.columns = df_pivot_g_nature.columns.droplevel(0)
df_pivot_g_nature = df_pivot_g_nature.rename_axis(None, axis=1)
df_pivot_g_nature = df_pivot_g_nature.reset_index()
category_n = (df_pivot_g_nature>0).sum()
category_n.to_csv(path_out1 + "category_n.csv", index=None)
category_n

Covered_Recipient_Profile_ID                               6882
Acquisitions                                                  3
Charitable Contribution                                       5
Consulting Fee                                             2253
Current or prospective ownership or investment interest       2
Debt forgiveness                                             24
Education                                                  5040
Entertainment                                                15
Food and Beverage                                          6758
Gift                                                        965
Grant                                                       102
Honoraria                                                   559
Long term medical supply or device loan                       6
Royalty or License                                           13
Travel and Lodging                                         2615
cme_speaking                            

# Import dataset

In [ ]:
usecols = ['Covered_Recipient_Profile_ID', 'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
           'Change_Type', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
           'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID', 'Total_Amount_of_Payment_USDollars',
           'Date_of_Payment', 'Number_of_Payments_Included_in_Total_Amount',
           'Nature_of_Payment_or_Transfer_of_Value', 'Dispute_Status_for_Publication',
           'Associated_Drug_or_Biological_NDC_1', 'Associated_Drug_or_Biological_NDC_2',
           'Associated_Drug_or_Biological_NDC_3', 'Associated_Drug_or_Biological_NDC_4',
           'Associated_Drug_or_Biological_NDC_5', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
           'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
           'Program_Year', 'Contextual_Information', "Record_ID"]

In [ ]:
dfg = pd.read_csv(BASE + '/' + specialty + "/general/full_general payments dataset2014-2022.csv",
                    low_memory=False, chunksize = 10000000, usecols= usecols)

In [ ]:
dfg = pd.concat((r for r in dfg), ignore_index=True)

In [ ]:
dfr = pd.read_csv(BASE + '/' + specialty + "/research/full_associated research payments dataset2014-2022_revision.csv",
                    low_memory=False)

In [ ]:
dfrd = pd.read_csv(BASE + '/' + specialty + "/research/full_direct research payments dataset2014-2022.csv",
                     low_memory=False)

In [ ]:
df_id = pd.read_csv(BASE + '/' + specialty + "/profile/matched physician profile data.csv",
                 low_memory=False )

In [ ]:
npi_specialty = pd.read_csv(BASE + '/' + specialty + "/profile/NPI physician profile data.csv",
                 low_memory=False )

In [ ]:
list_id = list(df_id["covered_recipient_profile_id"])